In [1]:
from model.trainfromcsv \
    import get_word_list, get_hiragana_list, get_word_yomi_list, read_file, add_vocab_from_files

In [2]:
from __future__ import print_function
from __future__ import division
from model.variationalmodel import VariationalModel
import random

import MeCab as mecab
import re
from operator import or_
from functools import reduce
import numpy as np

In [3]:
train_val = [0.1, 0.9]
train_path = ['./st-data/base.csv', './st-data/styled.csv']

In [4]:
parse_func = get_word_list

In [5]:
train_data = list(zip(*(reduce(lambda x, y: x + y,
                               map(lambda val_path:
                                   [[sent, val_path[0]]
                                    for sent in read_file(val_path[1], parse_func)],
                                   zip(train_val, train_path))))))
train_data_idx = random.sample(range(len(train_data[0])), len(train_data[0]))
train_data = list(map(lambda row: list(map(lambda idx: train_data[row][idx],
                                           train_data_idx)),
                      [0, 1]))
test_data_idx = random.sample(range(len(train_data[0])), len(train_data[0]) // 10)
test_data = list(map(lambda row: list(map(lambda idx: train_data[row][idx],
                                          test_data_idx)),
                     [0, 1]))

wyl = sorted(add_vocab_from_files(set(),
                                  ['./st-data/styled.csv', './st-data/base.csv'],
                                  parse_func))


In [6]:
""" Settings """
max_seq_length = 30
learning_rate = 1e-3
embedding_dim = 256
memory_dim = 512
log_dir = './output/logs/'

In [7]:
""" Information """
print('sentences size: {}'
      .format(len(train_data[0])))
print('vocaburaly size: {}'.format(len(wyl)))
for wy in wyl[:3]:
    print('vocab id: {} element: {}'.format(wyl.index(wy), wy))
print('example:')
for sentence, val in zip(train_data[0][:3], train_data[1][:3]):
    print('sentence: {} val: {}'.format(sentence, val))

""" create model """
model = VariationalModel(max_seq_length=max_seq_length, vocab=wyl, use_unknown=True,
                         learning_rate=learning_rate, embedding_dim=embedding_dim,
                         memory_dim=memory_dim,
                         use_sigmoid=True,
                         outcome_var=np.var(np.array(train_data[1])),
                         logdir=log_dir)


sentences size: 586
vocaburaly size: 437
vocab id: 0 element: 。
vocab id: 1 element: あ
vocab id: 2 element: あなた
example:
sentence: ['それ', 'は', 'お', '気の毒', 'に', '。'] val: 0.9
sentence: ['羊', 'の', '数', 'を', '数える', 'と', '良い', 'かも', '。'] val: 0.9
sentence: ['明日', 'は', '不安', 'だ', '。'] val: 0.9
logdir: ./output/logs/
Rescaling prediction MSE loss by outcome variance = 0.26
RNN type : GRU 
 cell_state_size : 512
Type of outcome prediction: nonlinear 
 All predictions in [0, 1] : True


In [12]:
def test_sentence(sentence, model):
    log_alpha = -1000
    init_seq = [get_word_yomi_list(sentence)]
    revision_results = \
        model.barrierGradientRevise(init_seq, log_alpha=log_alpha,
                                    outcome_opt_learn_rate=1.0,
                                    max_outcome_opt_iter=1000,
                                    use_adaptive=False)
    x_star, z_star, inferred_improvement, \
    outcome_star, reconstruct_init, z_init, \
    outcome_init, avg_sigma_init, edit_dist = revision_results
    print('{} -> {}'.format(init_seq[0], x_star))

In [9]:
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.0, invar_importanceval=0.0,
        max_epochs=200, invar_batchsize=32
    )

epoch: 0 train_loss=13.708134555816649 val_outcome_error=1.7112110696128677 val_reconstruction_error=2.1551724137931036
  avg_predicted_sigma=0.0016402225010097027
 avg_lik_loss=8.643686723709108 avg_pri_loss=67.26108169555664 avg_pred_loss=1.8569418549537655


epoch: 1 train_loss=13.288165187835695 val_outcome_error=0.9457667495894218 val_reconstruction_error=2.603448275862069
  avg_predicted_sigma=0.00138470612000674
 avg_lik_loss=8.37366919517517 avg_pri_loss=65.85084686279296 avg_pred_loss=1.7038102149963377


epoch: 2 train_loss=13.076792526245118 val_outcome_error=1.0927954822322334 val_reconstruction_error=2.1206896551724137
  avg_predicted_sigma=0.0014481628313660622
 avg_lik_loss=8.29686827659607 avg_pri_loss=64.41371994018554 avg_pred_loss=1.690126061439514


epoch: 3 train_loss=12.52940101623535 val_outcome_error=1.4570327913794943 val_reconstruction_error=2.310344827586207
  avg_predicted_sigma=0.0014693706762045622
 avg_lik_loss=7.892230463027954 avg_pri_loss=62.927092742919925 avg_pred_loss=1.5788511991500853


epoch: 4 train_loss=12.769987678527832 val_outcome_error=1.7217309284411224 val_reconstruction_error=2.603448275862069
  avg_predicted_sigma=0.001508914865553379
 avg_lik_loss=8.273239088058471 avg_pri_loss=61.259587860107416 avg_pred_loss=1.7778366982936857


...

epoch: 195 train_loss=7.494052982330322 val_outcome_error=6.77928925210569 val_reconstruction_error=0.6206896551724138
  avg_predicted_sigma=0.0005147275514900684
 avg_lik_loss=4.182634258270263 avg_pri_loss=38.984347152709965 avg_pred_loss=5.6026369571685795


epoch: 196 train_loss=6.584519529342651 val_outcome_error=6.131280629203681 val_reconstruction_error=0.7241379310344828
  avg_predicted_sigma=0.0003550458059180528
 avg_lik_loss=3.5091038703918453 avg_pri_loss=40.128666687011716 avg_pred_loss=3.636220932006836


epoch: 197 train_loss=7.617741060256959 val_outcome_error=6.0563178441382615 val_reconstruction_error=0.27586206896551724
  avg_predicted_sigma=0.0003428483905736357
 avg_lik_loss=3.5959070205688484 avg_pri_loss=40.4363956451416 avg_pred_loss=8.42631845474243


epoch: 198 train_loss=7.215859365463257 val_outcome_error=1.907420995967852 val_reconstruction_error=0.41379310344827586
  avg_predicted_sigma=0.0006168257095851004
 avg_lik_loss=3.583764433860779 avg_pri_loss=40.57392349243164 avg_pred_loss=6.330825638771057


epoch: 199 train_loss=6.42994966506958 val_outcome_error=1.715104043353974 val_reconstruction_error=0.3103448275862069
  avg_predicted_sigma=0.00035661825677379966
 avg_lik_loss=3.374737596511841 avg_pri_loss=40.49241104125976 avg_pred_loss=3.318598437309265


In [13]:
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.05, invar_importanceval=0.0,
        max_epochs=100, invar_batchsize=32
    )

epoch: 0 train_loss=8.039529991149902 val_outcome_error=4.336644116823785 val_reconstruction_error=0.3620689655172414
  avg_predicted_sigma=0.0005746067618019879
 avg_lik_loss=3.348311996459961 avg_pri_loss=39.444897460937504 avg_pred_loss=3.239766550064087


epoch: 1 train_loss=8.561498069763184 val_outcome_error=1.7170586643477852 val_reconstruction_error=0.29310344827586204
  avg_predicted_sigma=0.0004509196151047945
 avg_lik_loss=3.7584216833114628 avg_pri_loss=38.932011413574216 avg_pred_loss=4.291981983184814


epoch: 2 train_loss=8.016302156448365 val_outcome_error=1.787173897088313 val_reconstruction_error=1.0172413793103448
  avg_predicted_sigma=0.0006617344915866852
 avg_lik_loss=3.7084211111068726 avg_pri_loss=38.14188842773437 avg_pred_loss=2.269180130958557


epoch: 3 train_loss=7.7334000110626215 val_outcome_error=2.280710056393382 val_reconstruction_error=0.3103448275862069
  avg_predicted_sigma=0.0005532842478714883
 avg_lik_loss=3.4352294206619263 avg_pri_loss=37.24774932861329 avg_pred_loss=2.786357760429382


epoch: 4 train_loss=7.792588424682618 val_outcome_error=1.792283114721992 val_reconstruction_error=2.2241379310344827
  avg_predicted_sigma=0.0007344798650592566
 avg_lik_loss=3.545977234840394 avg_pri_loss=36.4043155670166 avg_pred_loss=3.145320677757263


...

epoch: 95 train_loss=5.496233367919922 val_outcome_error=2.259883075174963 val_reconstruction_error=0.1896551724137931
  avg_predicted_sigma=0.0007238035323098302
 avg_lik_loss=2.985889768600464 avg_pri_loss=23.973611068725585 avg_pred_loss=2.8677871704101556


epoch: 96 train_loss=5.221694660186767 val_outcome_error=2.6816357919077554 val_reconstruction_error=0.15517241379310345
  avg_predicted_sigma=0.0007331177475862205
 avg_lik_loss=2.7999579191207884 avg_pri_loss=23.841124153137205 avg_pred_loss=2.456420159339905


epoch: 97 train_loss=5.4231138706207265 val_outcome_error=2.163874535774343 val_reconstruction_error=0.05172413793103448
  avg_predicted_sigma=0.0006601927452720702
 avg_lik_loss=2.938065314292907 avg_pri_loss=23.57347755432129 avg_pred_loss=3.014626955986023


epoch: 98 train_loss=5.092080020904541 val_outcome_error=2.956097853308745 val_reconstruction_error=0.8103448275862069
  avg_predicted_sigma=0.0006809697952121496
 avg_lik_loss=2.63853919506073 avg_pri_loss=23.557437896728516 avg_pred_loss=2.7864398479461667


epoch: 99 train_loss=5.1681242942810055 val_outcome_error=2.11025516714135 val_reconstruction_error=0.8275862068965517
  avg_predicted_sigma=0.0006100464961491525
 avg_lik_loss=2.550613737106323 avg_pri_loss=23.616669654846195 avg_pred_loss=3.5766716003417964


In [15]:
model.save('./output/1016-2nd005.bin')

Model saved as: ./output/1016-2nd005.bin


In [16]:
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.10, invar_importanceval=0.0,
        max_epochs=200, invar_batchsize=32
    )

epoch: 0 train_loss=6.237641286849975 val_outcome_error=2.9505093091165495 val_reconstruction_error=0.29310344827586204
  avg_predicted_sigma=0.0006162177887745202
 avg_lik_loss=2.75883766412735 avg_pri_loss=23.63741073608399 avg_pred_loss=2.5119718432426454


epoch: 1 train_loss=6.358174848556518 val_outcome_error=2.61467751690219 val_reconstruction_error=0.20689655172413793
  avg_predicted_sigma=0.0006940244347788393
 avg_lik_loss=2.7698063373565676 avg_pri_loss=23.94726638793945 avg_pred_loss=2.7942498207092283


epoch: 2 train_loss=6.337757158279419 val_outcome_error=2.7380113343058796 val_reconstruction_error=0.3275862068965517
  avg_predicted_sigma=0.0009946194477379322
 avg_lik_loss=2.7972044229507445 avg_pri_loss=24.04659194946289 avg_pred_loss=2.453960859775543


epoch: 3 train_loss=6.458367109298707 val_outcome_error=2.9113590412161288 val_reconstruction_error=0.1724137931034483
  avg_predicted_sigma=0.0007168658194132149
 avg_lik_loss=2.988856387138367 avg_pri_loss=24.094309997558593 avg_pred_loss=2.091560757160187


epoch: 4 train_loss=6.301677179336548 val_outcome_error=1.7687635440493747 val_reconstruction_error=0.5862068965517241
  avg_predicted_sigma=0.0010529562132433057
 avg_lik_loss=2.835154724121094 avg_pri_loss=23.92627353668213 avg_pred_loss=2.191783440113068


...

epoch: 195 train_loss=7.85364031791687 val_outcome_error=1.9268908716025546 val_reconstruction_error=0.7586206896551724
  avg_predicted_sigma=0.0006517576985061169
 avg_lik_loss=3.3843561887741087 avg_pri_loss=29.636982917785645 avg_pred_loss=2.1978933930397035


epoch: 196 train_loss=7.6438933372497555 val_outcome_error=1.099934114203563 val_reconstruction_error=0.5344827586206896
  avg_predicted_sigma=0.0007752643432468176
 avg_lik_loss=3.147238230705261 avg_pri_loss=29.491044425964358 avg_pred_loss=2.415435993671418


epoch: 197 train_loss=7.643420600891114 val_outcome_error=1.5307963898827357 val_reconstruction_error=2.0689655172413794
  avg_predicted_sigma=0.0007589144515804946
 avg_lik_loss=3.345546364784241 avg_pri_loss=28.98926849365234 avg_pred_loss=1.902837425470352


epoch: 198 train_loss=7.329139757156373 val_outcome_error=1.5757541155131578 val_reconstruction_error=0.8103448275862069
  avg_predicted_sigma=0.0006434273673221469
 avg_lik_loss=2.9266302108764646 avg_pri_loss=28.25369930267334 avg_pred_loss=3.0238268613815307


epoch: 199 train_loss=7.489665222167969 val_outcome_error=4.716958597979106 val_reconstruction_error=0.1896551724137931
  avg_predicted_sigma=0.0006929097580723464
 avg_lik_loss=3.149536108970642 avg_pri_loss=27.59628791809082 avg_pred_loss=3.376928567886353


In [17]:
model.save('./output/1016-2nd010.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.15, invar_importanceval=0.0,
        max_epochs=100, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd010.bin


epoch: 0 train_loss=8.49648380279541 val_outcome_error=0.4421150226839182 val_reconstruction_error=0.6551724137931034
  avg_predicted_sigma=0.0005948340403847396
 avg_lik_loss=3.230298709869385 avg_pri_loss=27.261530303955077 avg_pred_loss=2.1423679351806637


epoch: 1 train_loss=8.44307107925415 val_outcome_error=0.5822837700955548 val_reconstruction_error=0.9827586206896551
  avg_predicted_sigma=0.0005003191763535142
 avg_lik_loss=3.4719823598861694 avg_pri_loss=26.602371597290038 avg_pred_loss=1.4352361381053926


epoch: 2 train_loss=7.945751953124999 val_outcome_error=1.1703084733991194 val_reconstruction_error=0.3620689655172414
  avg_predicted_sigma=0.0007033082074485719
 avg_lik_loss=3.2540324449539186 avg_pri_loss=25.461249923706053 avg_pred_loss=1.2471699357032777


epoch: 3 train_loss=7.337952661514282 val_outcome_error=0.7530218010539143 val_reconstruction_error=0.3448275862068966
  avg_predicted_sigma=0.0004644300206564367
 avg_lik_loss=2.9055656671524046 avg_pri_loss=24.19355010986328 avg_pred_loss=1.2825464010238645


...

epoch: 95 train_loss=12.992680549621584 val_outcome_error=3.3785910270924684 val_reconstruction_error=3.1551724137931036
  avg_predicted_sigma=0.0004849751712754369
 avg_lik_loss=4.389267492294311 avg_pri_loss=35.535430908203125 avg_pred_loss=10.190553951263428


epoch: 96 train_loss=11.802182960510253 val_outcome_error=1.7311216580102657 val_reconstruction_error=5.827586206896552
  avg_predicted_sigma=0.0005980730056762695
 avg_lik_loss=4.073929953575135 avg_pri_loss=37.02107238769531 avg_pred_loss=3.8372219443321227


epoch: 97 train_loss=11.698613643646238 val_outcome_error=2.7396640652492463 val_reconstruction_error=1.2241379310344827
  avg_predicted_sigma=0.0008124540327116847
 avg_lik_loss=3.993699026107788 avg_pri_loss=37.34037399291992 avg_pred_loss=3.31931939125061


epoch: 98 train_loss=11.333606624603274 val_outcome_error=2.096865118271643 val_reconstruction_error=1.0344827586206897
  avg_predicted_sigma=0.0009626945247873664
 avg_lik_loss=3.7358721256256104 avg_pri_loss=36.12237854003906 avg_pred_loss=4.102362859249116


epoch: 99 train_loss=10.389750099182129 val_outcome_error=2.9065766516698037 val_reconstruction_error=2.413793103448276
  avg_predicted_sigma=0.0009792528580874205
 avg_lik_loss=3.3857729434967045 avg_pri_loss=34.748300552368164 avg_pred_loss=2.516535663604736


In [18]:
# model.save('./output/1016-2nd010.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.15, invar_importanceval=0.0,
        max_epochs=100, invar_batchsize=32
    )

epoch: 0 train_loss=9.72632761001587 val_outcome_error=3.3478360695535354 val_reconstruction_error=1.5862068965517242
  avg_predicted_sigma=0.0010124024702236056
 avg_lik_loss=3.1617111921310426 avg_pri_loss=33.080528259277344 avg_pred_loss=2.101213479042053


epoch: 1 train_loss=9.668736362457276 val_outcome_error=0.9567889856609 val_reconstruction_error=1.706896551724138
  avg_predicted_sigma=0.0009131429833360016
 avg_lik_loss=3.429540967941284 avg_pri_loss=31.808802032470698 avg_pred_loss=1.9526686429977418


epoch: 2 train_loss=8.886857414245606 val_outcome_error=0.8939531136230955 val_reconstruction_error=1.1551724137931034
  avg_predicted_sigma=0.0012255600886419415
 avg_lik_loss=3.0829832553863525 avg_pri_loss=30.58997116088867 avg_pred_loss=1.0094567656517028


epoch: 3 train_loss=8.65859432220459 val_outcome_error=1.2355633003342925 val_reconstruction_error=3.603448275862069
  avg_predicted_sigma=0.0015825476730242372
 avg_lik_loss=3.153169465065002 avg_pri_loss=29.26440372467041 avg_pred_loss=1.0105649083852768


epoch: 4 train_loss=8.577447843551635 val_outcome_error=1.7759865295937207 val_reconstruction_error=2.3448275862068964
  avg_predicted_sigma=0.0010659570107236505
 avg_lik_loss=3.342876863479614 avg_pri_loss=27.980636596679688 avg_pred_loss=1.148357230424881


...

epoch: 95 train_loss=7.3611075401306145 val_outcome_error=3.2234718315488444 val_reconstruction_error=0.6551724137931034
  avg_predicted_sigma=0.0006583669455721974
 avg_lik_loss=3.1673517465591434 avg_pri_loss=19.89008159637451 avg_pred_loss=5.104921984672546


epoch: 96 train_loss=7.275874519348145 val_outcome_error=6.077165396464938 val_reconstruction_error=0.4827586206896552
  avg_predicted_sigma=0.00027983912150375545
 avg_lik_loss=2.9751127004623417 avg_pri_loss=20.989132308959963 avg_pred_loss=4.339597225189209


epoch: 97 train_loss=7.344488620758056 val_outcome_error=3.641334453090465 val_reconstruction_error=1.1206896551724137
  avg_predicted_sigma=0.0005209756200201809
 avg_lik_loss=2.924642133712768 avg_pri_loss=21.798504829406742 avg_pred_loss=4.015321063995361


epoch: 98 train_loss=7.483699750900269 val_outcome_error=4.390236560953194 val_reconstruction_error=2.293103448275862
  avg_predicted_sigma=0.0006684708059765399
 avg_lik_loss=3.040015697479248 avg_pri_loss=22.380538940429688 avg_pred_loss=3.49029347896576


epoch: 99 train_loss=6.819330406188965 val_outcome_error=2.501782356523217 val_reconstruction_error=0.603448275862069
  avg_predicted_sigma=0.0007596587529405951
 avg_lik_loss=2.5926995038986203 avg_pri_loss=22.415101051330563 avg_pred_loss=2.1998693764209745


In [19]:
model.save('./output/1016-2nd015.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.20, invar_importanceval=0.0,
        max_epochs=100, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd015.bin


epoch: 0 train_loss=7.538876247406006 val_outcome_error=1.9576315913334945 val_reconstruction_error=1.3620689655172413
  avg_predicted_sigma=0.0005228471709415317
 avg_lik_loss=2.5806270837783813 avg_pri_loss=21.963078117370603 avg_pred_loss=1.3137449502944947


epoch: 1 train_loss=7.854285573959349 val_outcome_error=1.594716161465807 val_reconstruction_error=0.9482758620689655
  avg_predicted_sigma=0.0008392013842239976
 avg_lik_loss=3.128613638877869 avg_pri_loss=21.328951835632328 avg_pred_loss=1.1187047481536865


epoch: 2 train_loss=7.601318407058716 val_outcome_error=0.8768493062790993 val_reconstruction_error=1.0
  avg_predicted_sigma=0.001307969680055976
 avg_lik_loss=3.1251797914505004 avg_pri_loss=20.674691200256348 avg_pred_loss=0.7223468810319901


epoch: 3 train_loss=7.3346119880676275 val_outcome_error=1.3196860021137409 val_reconstruction_error=0.20689655172413793
  avg_predicted_sigma=0.001048558740876615
 avg_lik_loss=3.025836777687073 avg_pri_loss=20.058008384704593 avg_pred_loss=0.6761987335979939


...

epoch: 95 train_loss=7.055755186080933 val_outcome_error=1.9635551789480379 val_reconstruction_error=0.39655172413793105
  avg_predicted_sigma=0.0010987331625074148
 avg_lik_loss=2.8610491037368773 avg_pri_loss=18.520969200134275 avg_pred_loss=2.152287349104881


epoch: 96 train_loss=7.241331577301025 val_outcome_error=3.6266279952486893 val_reconstruction_error=3.9310344827586206
  avg_predicted_sigma=0.0015849149785935879
 avg_lik_loss=2.8238013744354244 avg_pri_loss=18.57284717559814 avg_pred_loss=3.232029694318771


epoch: 97 train_loss=7.617864036560059 val_outcome_error=4.897378844041015 val_reconstruction_error=2.206896551724138
  avg_predicted_sigma=0.0014402552042156458
 avg_lik_loss=3.0697415590286257 avg_pri_loss=18.89354705810547 avg_pred_loss=3.5256328582763667


epoch: 98 train_loss=7.372441291809081 val_outcome_error=1.6584571908981465 val_reconstruction_error=0.8103448275862069
  avg_predicted_sigma=0.0014357054606080055
 avg_lik_loss=2.8983900308609 avg_pri_loss=19.18482284545899 avg_pred_loss=2.6962375044822693


epoch: 99 train_loss=7.006415700912475 val_outcome_error=1.5242870085133513 val_reconstruction_error=0.10344827586206896
  avg_predicted_sigma=0.0012918435968458652
 avg_lik_loss=2.745982432365418 avg_pri_loss=19.2481502532959 avg_pred_loss=1.4593752443790435


In [20]:
model.save('./output/1016-2nd015.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.20, invar_importanceval=0.0,
        max_epochs=100, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd015.bin


epoch: 0 train_loss=7.054725074768067 val_outcome_error=1.2112571666048508 val_reconstruction_error=0.7758620689655172
  avg_predicted_sigma=0.0008226020727306604
 avg_lik_loss=2.8904967546463016 avg_pri_loss=19.172575759887696 avg_pred_loss=1.1043963253498077


epoch: 1 train_loss=7.212964010238647 val_outcome_error=1.3691451793243372 val_reconstruction_error=2.310344827586207
  avg_predicted_sigma=0.0014639325672760606
 avg_lik_loss=3.1019078254699703 avg_pri_loss=18.768957901000977 avg_pred_loss=1.437750443816185


epoch: 2 train_loss=7.454967355728149 val_outcome_error=0.6648131423238126 val_reconstruction_error=0.017241379310344827
  avg_predicted_sigma=0.0012542631011456251
 avg_lik_loss=3.4731848478317255 avg_pri_loss=18.394719314575198 avg_pred_loss=1.374281281232834


epoch: 3 train_loss=6.700004434585572 val_outcome_error=1.1248798757109224 val_reconstruction_error=0.39655172413793105
  avg_predicted_sigma=0.0012052818201482296
 avg_lik_loss=2.7771321415901182 avg_pri_loss=18.095895195007323 avg_pred_loss=1.307344686985016


...

epoch: 95 train_loss=7.019394397735595 val_outcome_error=0.48454077589931305 val_reconstruction_error=0.1896551724137931
  avg_predicted_sigma=0.001177431084215641
 avg_lik_loss=2.852177000045776 avg_pri_loss=19.32347679138184 avg_pred_loss=0.901239800453186


epoch: 96 train_loss=7.040259933471679 val_outcome_error=1.1644794723644487 val_reconstruction_error=2.8448275862068964
  avg_predicted_sigma=0.0013205177383497357
 avg_lik_loss=2.8614186525344842 avg_pri_loss=19.02652778625488 avg_pred_loss=1.368574211001396


epoch: 97 train_loss=7.028721904754638 val_outcome_error=1.4248702525585328 val_reconstruction_error=1.5517241379310345
  avg_predicted_sigma=0.0016003745840862393
 avg_lik_loss=2.6825178623199464 avg_pri_loss=18.65666675567627 avg_pred_loss=2.7011330962181086


epoch: 98 train_loss=7.82270255088806 val_outcome_error=2.5196243445862483 val_reconstruction_error=0.20689655172413793
  avg_predicted_sigma=0.001194389071315527
 avg_lik_loss=2.977216434478759 avg_pri_loss=19.94879360198975 avg_pred_loss=3.5936386585235596


epoch: 99 train_loss=8.42531714439392 val_outcome_error=1.3150665039203162 val_reconstruction_error=2.3275862068965516
  avg_predicted_sigma=0.0008799447095952928
 avg_lik_loss=2.980682635307312 avg_pri_loss=22.608640670776364 avg_pred_loss=3.04729346036911


In [21]:
# model.save('./output/1016-2nd020.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.20, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=8.084732675552369 val_outcome_error=0.4429317594080327 val_reconstruction_error=0.603448275862069
  avg_predicted_sigma=0.0006841812864877284
 avg_lik_loss=2.7562649488449096 avg_pri_loss=23.585477638244633 avg_pred_loss=1.0352926433086396


epoch: 1 train_loss=7.927456569671632 val_outcome_error=0.3412580355705536 val_reconstruction_error=3.103448275862069
  avg_predicted_sigma=0.00027945052715949714
 avg_lik_loss=2.689599943161011 avg_pri_loss=23.612292098999024 avg_pred_loss=0.518277943879366


epoch: 2 train_loss=8.11482677459717 val_outcome_error=0.4387280768437074 val_reconstruction_error=3.293103448275862
  avg_predicted_sigma=0.0008492899360135198
 avg_lik_loss=3.0493245124816895 avg_pri_loss=23.015795516967774 avg_pred_loss=0.5409248165786267


epoch: 3 train_loss=7.709885168075561 val_outcome_error=0.47033221024738775 val_reconstruction_error=0.15517241379310345
  avg_predicted_sigma=0.000457163667306304
 avg_lik_loss=2.866041874885559 avg_pri_loss=22.212743949890136 avg_pred_loss=0.4472731053829192


epoch: 4 train_loss=7.290707635879516 val_outcome_error=0.4216237171599603 val_reconstruction_error=0.0
  avg_predicted_sigma=0.0006817506509833038
 avg_lik_loss=2.651250982284546 avg_pri_loss=21.42639045715332 avg_pred_loss=0.41284673362970353


...

epoch: 45 train_loss=6.187152910232544 val_outcome_error=0.6054255719053392 val_reconstruction_error=1.6551724137931034
  avg_predicted_sigma=0.0008781403303146362
 avg_lik_loss=2.7291743993759154 avg_pri_loss=16.94680347442627 avg_pred_loss=0.4807891607284546


epoch: 46 train_loss=6.1096282482147215 val_outcome_error=0.7153854626915231 val_reconstruction_error=0.9482758620689655
  avg_predicted_sigma=0.0009552495903335512
 avg_lik_loss=2.7194695472717285 avg_pri_loss=16.677331733703614 avg_pred_loss=0.49760030359029755


epoch: 47 train_loss=5.888894891738892 val_outcome_error=0.5387370110116388 val_reconstruction_error=0.9137931034482759
  avg_predicted_sigma=0.0009557171142660081
 avg_lik_loss=2.4911974430084225 avg_pri_loss=16.446107101440433 avg_pred_loss=0.7976077884435654


epoch: 48 train_loss=6.185227680206299 val_outcome_error=1.690685553050535 val_reconstruction_error=2.3448275862068964
  avg_predicted_sigma=0.0007119557121768594
 avg_lik_loss=2.680760836601258 avg_pri_loss=16.473200988769534 avg_pred_loss=1.369958934187889


epoch: 49 train_loss=6.112629508972167 val_outcome_error=0.8006788191716845 val_reconstruction_error=1.8620689655172413
  avg_predicted_sigma=0.0007947488338686526
 avg_lik_loss=2.6415338039398195 avg_pri_loss=16.42365493774414 avg_pred_loss=1.2551069557666776


In [22]:
# model.save('./output/1016-2nd020.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.20, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=6.049569368362428 val_outcome_error=0.46445146296465806 val_reconstruction_error=0.8448275862068966
  avg_predicted_sigma=0.0005999127752147615
 avg_lik_loss=2.651471865177155 avg_pri_loss=16.413005447387693 avg_pred_loss=0.8963056564331056


epoch: 1 train_loss=6.03399510383606 val_outcome_error=0.27100492552870964 val_reconstruction_error=0.6724137931034483
  avg_predicted_sigma=0.0008546714088879526
 avg_lik_loss=2.700676774978638 avg_pri_loss=16.423749542236326 avg_pred_loss=0.5574518829584122


epoch: 2 train_loss=5.988440895080567 val_outcome_error=0.6564004216612175 val_reconstruction_error=0.41379310344827586
  avg_predicted_sigma=0.0008999482961371541
 avg_lik_loss=2.6619978189468383 avg_pri_loss=16.434922981262208 avg_pred_loss=0.4966925606131553


epoch: 3 train_loss=6.148295640945435 val_outcome_error=1.1426941215242565 val_reconstruction_error=0.3275862068965517
  avg_predicted_sigma=0.00046530732652172446
 avg_lik_loss=2.834402465820313 avg_pri_loss=16.379146957397463 avg_pred_loss=0.5609065353870393


epoch: 4 train_loss=6.118927383422852 val_outcome_error=0.8058817173627101 val_reconstruction_error=0.4827586206896552
  avg_predicted_sigma=0.0003728463198058307
 avg_lik_loss=2.7487447261810303 avg_pri_loss=16.306181812286376 avg_pred_loss=0.9345341578125953


...

epoch: 45 train_loss=5.778621196746826 val_outcome_error=1.5804698623703974 val_reconstruction_error=0.1896551724137931
  avg_predicted_sigma=0.0007937639602459967
 avg_lik_loss=2.539055156707764 avg_pri_loss=15.658614063262936 avg_pred_loss=1.0814976304769515


epoch: 46 train_loss=6.083599662780761 val_outcome_error=0.6528639916148624 val_reconstruction_error=0.1724137931034483
  avg_predicted_sigma=0.0008906784933060408
 avg_lik_loss=2.749672794342041 avg_pri_loss=15.603097820281983 avg_pred_loss=1.6998378440737725


epoch: 47 train_loss=5.84108052253723 val_outcome_error=1.0301371772836243 val_reconstruction_error=0.034482758620689655
  avg_predicted_sigma=0.0006703183171339333
 avg_lik_loss=2.645583081245422 avg_pri_loss=15.763675022125245 avg_pred_loss=0.7327020078897476


epoch: 48 train_loss=6.001976251602174 val_outcome_error=0.8992268061663728 val_reconstruction_error=0.896551724137931
  avg_predicted_sigma=0.0007844166248105466
 avg_lik_loss=2.6889220237731934 avg_pri_loss=15.864156341552734 avg_pred_loss=1.219286760687828


epoch: 49 train_loss=6.107541036605835 val_outcome_error=1.3096112553320205 val_reconstruction_error=2.2413793103448274
  avg_predicted_sigma=0.0010408784728497267
 avg_lik_loss=2.7936834096908574 avg_pri_loss=15.949594593048094 avg_pred_loss=1.134360608458519


In [23]:
model.save('./output/1016-2nd020.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.25, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd020.bin


epoch: 0 train_loss=6.578854846954346 val_outcome_error=0.4607360965012421 val_reconstruction_error=1.6551724137931034
  avg_predicted_sigma=0.0010999685619026423
 avg_lik_loss=2.6288982868194584 avg_pri_loss=15.88234634399414 avg_pred_loss=0.6271897494792938


epoch: 1 train_loss=6.732159948349 val_outcome_error=0.6818730338562318 val_reconstruction_error=1.1379310344827587
  avg_predicted_sigma=0.0011536458041518927
 avg_lik_loss=2.8336748361587527 avg_pri_loss=15.679648208618165 avg_pred_loss=0.7355158984661102


epoch: 2 train_loss=7.0991254329681395 val_outcome_error=1.1900320396868538 val_reconstruction_error=2.8448275862068964
  avg_predicted_sigma=0.0011506386799737811
 avg_lik_loss=3.2203006267547605 avg_pri_loss=15.58017635345459 avg_pred_loss=0.8944834768772126


epoch: 3 train_loss=7.167628574371338 val_outcome_error=0.7681811576379303 val_reconstruction_error=0.06896551724137931
  avg_predicted_sigma=0.0006606562528759241
 avg_lik_loss=3.2845347642898552 avg_pri_loss=15.620088863372803 avg_pred_loss=0.8686482310295106


...

epoch: 45 train_loss=7.118335580825805 val_outcome_error=0.41653089748337263 val_reconstruction_error=0.7241379310344828
  avg_predicted_sigma=0.0011152756633237004
 avg_lik_loss=2.9434972524642946 avg_pri_loss=16.75782413482666 avg_pred_loss=0.47224640622735026


epoch: 46 train_loss=7.118704080581664 val_outcome_error=1.3222450972827553 val_reconstruction_error=0.0
  avg_predicted_sigma=0.0010544180404394865
 avg_lik_loss=2.9972402095794677 avg_pri_loss=16.50380249023437 avg_pred_loss=0.617034749686718


epoch: 47 train_loss=6.941134452819824 val_outcome_error=0.30427015400460933 val_reconstruction_error=0.8275862068965517
  avg_predicted_sigma=0.0012763591948896646
 avg_lik_loss=2.825412702560425 avg_pri_loss=16.204040336608887 avg_pred_loss=1.0239824384450913


epoch: 48 train_loss=6.713790559768675 val_outcome_error=0.34518601970867785 val_reconstruction_error=2.0
  avg_predicted_sigma=0.0011165101313963532
 avg_lik_loss=2.771662735939026 avg_pri_loss=15.96944065093994 avg_pred_loss=0.4818280264735221


epoch: 49 train_loss=6.858241033554077 val_outcome_error=0.43080594453094084 val_reconstruction_error=4.137931034482759
  avg_predicted_sigma=0.001533004455268383
 avg_lik_loss=2.9730748176574706 avg_pri_loss=15.80955181121826 avg_pred_loss=0.49205886796116843


In [24]:
model.save('./output/1016-2nd025.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.30, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd025.bin


epoch: 0 train_loss=7.209211921691895 val_outcome_error=0.758711706108217 val_reconstruction_error=0.43103448275862066
  avg_predicted_sigma=0.0014769569970667362
 avg_lik_loss=2.6196997642517093 avg_pri_loss=15.602783584594727 avg_pred_loss=0.5973173320293427


epoch: 1 train_loss=7.199833297729491 val_outcome_error=4.045914753804783 val_reconstruction_error=0.6896551724137931
  avg_predicted_sigma=0.0012102102627977729
 avg_lik_loss=2.5828327178955077 avg_pri_loss=15.396349239349362 avg_pred_loss=1.11360564827919


epoch: 2 train_loss=8.145923280715943 val_outcome_error=2.107409822838784 val_reconstruction_error=1.6724137931034482
  avg_predicted_sigma=0.0018770748283714056
 avg_lik_loss=2.7222304821014403 avg_pri_loss=15.956182384490965 avg_pred_loss=4.319853079319


epoch: 3 train_loss=8.882962703704834 val_outcome_error=2.437171569499054 val_reconstruction_error=1.1551724137931034
  avg_predicted_sigma=0.0006386765744537115
 avg_lik_loss=3.1087961196899414 avg_pri_loss=17.92286930084229 avg_pred_loss=2.651121184229851


...

epoch: 45 train_loss=7.210793209075928 val_outcome_error=2.2948466015956903 val_reconstruction_error=1.2586206896551724
  avg_predicted_sigma=0.0004986487911082804
 avg_lik_loss=2.6267673730850225 avg_pri_loss=15.114949512481688 avg_pred_loss=1.4901404812932015


epoch: 46 train_loss=7.467573356628418 val_outcome_error=2.583107976988201 val_reconstruction_error=1.293103448275862
  avg_predicted_sigma=0.00040121632628142834
 avg_lik_loss=2.669033408164978 avg_pri_loss=15.28380060195923 avg_pred_loss=2.3034734368324283


epoch: 47 train_loss=7.904256534576416 val_outcome_error=2.3022739846958458 val_reconstruction_error=1.8448275862068966
  avg_predicted_sigma=0.00032485579140484333
 avg_lik_loss=2.8200555086135863 avg_pri_loss=16.015719890594482 avg_pred_loss=2.4905480504035946


epoch: 48 train_loss=8.253158235549927 val_outcome_error=1.4577708448425217 val_reconstruction_error=1.5
  avg_predicted_sigma=0.0005430096643976867
 avg_lik_loss=2.7808194875717165 avg_pri_loss=17.271566581726074 avg_pred_loss=2.2056327685713764


epoch: 49 train_loss=8.388883924484253 val_outcome_error=2.2253029614212996 val_reconstruction_error=0.8103448275862069
  avg_predicted_sigma=0.0005229550297372043
 avg_lik_loss=2.779087901115418 avg_pri_loss=17.808630561828615 avg_pred_loss=1.936365211009979


In [25]:
model.save('./output/1016-2nd030.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.35, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd030.bin


epoch: 0 train_loss=8.947319221496581 val_outcome_error=0.8606682587970201 val_reconstruction_error=1.3103448275862069
  avg_predicted_sigma=0.0005903554265387356
 avg_lik_loss=2.5977479696273806 avg_pri_loss=17.690433692932128 avg_pred_loss=1.6882917374372481


epoch: 1 train_loss=9.151019763946532 val_outcome_error=1.9408444232161892 val_reconstruction_error=3.913793103448276
  avg_predicted_sigma=0.0004302546149119735
 avg_lik_loss=2.805148649215698 avg_pri_loss=17.366848182678222 avg_pred_loss=2.38941011428833


epoch: 2 train_loss=9.127735805511472 val_outcome_error=0.5748628850087943 val_reconstruction_error=5.293103448275862
  avg_predicted_sigma=0.000672897498589009
 avg_lik_loss=3.0512661218643187 avg_pri_loss=17.03655223846436 avg_pred_loss=1.7298830926418303


epoch: 3 train_loss=8.979938316345216 val_outcome_error=2.40876474979987 val_reconstruction_error=0.5
  avg_predicted_sigma=0.0006049805087968707
 avg_lik_loss=3.005628752708435 avg_pri_loss=16.866064643859865 avg_pred_loss=1.5335692971944808


...

epoch: 45 train_loss=8.214229106903076 val_outcome_error=4.7552133057622825 val_reconstruction_error=2.6379310344827585
  avg_predicted_sigma=0.00013083283556625247
 avg_lik_loss=2.8895434856414797 avg_pri_loss=15.360962677001954 avg_pred_loss=1.2067152082920076


epoch: 46 train_loss=8.147508668899537 val_outcome_error=1.223443053604558 val_reconstruction_error=3.1379310344827585
  avg_predicted_sigma=0.00014279021706897765
 avg_lik_loss=2.6516030073165897 avg_pri_loss=15.648701000213622 avg_pred_loss=1.4545364752411842


epoch: 47 train_loss=8.35014386177063 val_outcome_error=0.6039522991777392 val_reconstruction_error=2.086206896551724
  avg_predicted_sigma=0.00015242621884681284
 avg_lik_loss=2.9289224147796635 avg_pri_loss=15.877107524871827 avg_pred_loss=0.6674297668039798


epoch: 48 train_loss=8.285386419296266 val_outcome_error=1.3801514878329026 val_reconstruction_error=3.0
  avg_predicted_sigma=0.0005024059792049229
 avg_lik_loss=2.826311469078064 avg_pri_loss=15.815641880035399 avg_pred_loss=0.9667205184698107


epoch: 49 train_loss=8.45968337059021 val_outcome_error=4.551868068109458 val_reconstruction_error=0.6206896551724138
  avg_predicted_sigma=0.00027090319781564176
 avg_lik_loss=2.6675116777420045 avg_pri_loss=15.659920883178712 avg_pred_loss=2.973241382837296


In [26]:
model.save('./output/1016-2nd035.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.40, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd035.bin


epoch: 0 train_loss=9.092032527923584 val_outcome_error=0.6378744394081551 val_reconstruction_error=0.46551724137931033
  avg_predicted_sigma=0.0002620763552840799
 avg_lik_loss=2.6208020925521853 avg_pri_loss=16.108822250366213 avg_pred_loss=1.6653788447380065


epoch: 1 train_loss=9.165835571289064 val_outcome_error=0.4371965227816776 val_reconstruction_error=0.7068965517241379
  avg_predicted_sigma=0.0008383764652535319
 avg_lik_loss=2.659137511253357 avg_pri_loss=16.54076328277588 avg_pred_loss=0.872466391324997


epoch: 2 train_loss=9.215792655944824 val_outcome_error=1.0784671091100173 val_reconstruction_error=1.9655172413793103
  avg_predicted_sigma=0.0003645972174126655
 avg_lik_loss=2.6118257999420167 avg_pri_loss=16.78566589355469 avg_pred_loss=0.8077294707298279


epoch: 3 train_loss=9.603804969787596 val_outcome_error=1.0377017818116718 val_reconstruction_error=0.5344827586206896
  avg_predicted_sigma=0.0007005904917605221
 avg_lik_loss=2.995452165603638 avg_pri_loss=16.806484031677243 avg_pred_loss=0.887281584739685


...

epoch: 45 train_loss=9.437099456787108 val_outcome_error=4.162991596770355 val_reconstruction_error=0.5689655172413793
  avg_predicted_sigma=0.0014395107282325625
 avg_lik_loss=2.580589509010315 avg_pri_loss=16.753314208984378 avg_pred_loss=2.173895448446274


epoch: 46 train_loss=11.086923313140867 val_outcome_error=2.897938782526464 val_reconstruction_error=3.8793103448275863
  avg_predicted_sigma=0.001068117213435471
 avg_lik_loss=3.0459734439849853 avg_pri_loss=17.896677207946777 avg_pred_loss=5.854126989841462


epoch: 47 train_loss=11.60605878829956 val_outcome_error=2.154170126103374 val_reconstruction_error=0.9137931034482759
  avg_predicted_sigma=0.0008872128091752529
 avg_lik_loss=3.1366693973541255 avg_pri_loss=19.998794555664062 avg_pred_loss=3.332207989692688


epoch: 48 train_loss=12.079580593109132 val_outcome_error=2.786200199856496 val_reconstruction_error=2.689655172413793
  avg_predicted_sigma=0.0012890652287751436
 avg_lik_loss=3.210442781448364 avg_pri_loss=20.94471645355225 avg_pred_loss=3.2804394721984864


epoch: 49 train_loss=11.950604248046872 val_outcome_error=1.1546503193330016 val_reconstruction_error=3.8448275862068964
  avg_predicted_sigma=0.0006877761916257441
 avg_lik_loss=2.826685285568237 avg_pri_loss=21.387871742248535 avg_pred_loss=3.5029688060283655


In [27]:
# model.save('./output/1016-2nd040.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.40, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=11.812639904022214 val_outcome_error=2.6361609831381605 val_reconstruction_error=1.8448275862068966
  avg_predicted_sigma=0.0008061257540248334
 avg_lik_loss=2.7496646165847776 avg_pri_loss=21.64692268371582 avg_pred_loss=2.5787434816360477


epoch: 1 train_loss=11.652837848663331 val_outcome_error=3.18522728841503 val_reconstruction_error=1.6379310344827587
  avg_predicted_sigma=0.0011580938007682562
 avg_lik_loss=2.868932795524597 avg_pri_loss=21.553771209716793 avg_pred_loss=1.369021064043045


epoch: 2 train_loss=11.311215019226074 val_outcome_error=1.8163350374144065 val_reconstruction_error=1.0344827586206897
  avg_predicted_sigma=0.0010358800645917654
 avg_lik_loss=2.911621522903442 avg_pri_loss=20.64294147491455 avg_pred_loss=1.4514297157526015


epoch: 3 train_loss=10.860607433319093 val_outcome_error=0.6867273245162185 val_reconstruction_error=1.0172413793103448
  avg_predicted_sigma=0.0009188445401377976
 avg_lik_loss=3.0431742906570434 avg_pri_loss=19.533700752258305 avg_pred_loss=0.9834254086017608


epoch: 4 train_loss=10.332449340820313 val_outcome_error=0.3420708786952491 val_reconstruction_error=6.4655172413793105
  avg_predicted_sigma=0.0009786227019503713
 avg_lik_loss=2.8628235816955563 avg_pri_loss=18.65949077606201 avg_pred_loss=1.1143919855356217


...

epoch: 45 train_loss=9.50804958343506 val_outcome_error=1.1633393453552772 val_reconstruction_error=2.9482758620689653
  avg_predicted_sigma=0.0018771275645121932
 avg_lik_loss=2.8151302099227906 avg_pri_loss=16.78224830627441 avg_pred_loss=1.306631350517273


epoch: 46 train_loss=9.473145103454591 val_outcome_error=0.6936559741497542 val_reconstruction_error=0.4827586206896552
  avg_predicted_sigma=0.0020308925304561853
 avg_lik_loss=2.6892849922180178 avg_pri_loss=16.79614715576172 avg_pred_loss=1.7151076853275298


epoch: 47 train_loss=9.695108222961426 val_outcome_error=0.5145753945387582 val_reconstruction_error=1.8275862068965518
  avg_predicted_sigma=0.0018785464344546199
 avg_lik_loss=2.93801851272583 avg_pri_loss=16.829775238037108 avg_pred_loss=1.563450187444687


epoch: 48 train_loss=9.793875408172608 val_outcome_error=1.5783303755699973 val_reconstruction_error=2.5689655172413794
  avg_predicted_sigma=0.0013994722394272685
 avg_lik_loss=3.001025652885437 avg_pri_loss=17.12419681549072 avg_pred_loss=1.0998620793223381


epoch: 49 train_loss=9.328704071044923 val_outcome_error=0.5734472695137715 val_reconstruction_error=1.3275862068965518
  avg_predicted_sigma=0.0011690520914271474
 avg_lik_loss=2.567648530006409 avg_pri_loss=17.176531982421878 avg_pred_loss=0.7132916733622551


In [28]:
model.save('./output/1016-2nd040.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.45, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd040.bin


epoch: 0 train_loss=10.250150203704834 val_outcome_error=0.6787646605590846 val_reconstruction_error=1.793103448275862
  avg_predicted_sigma=0.0011705541983246803
 avg_lik_loss=2.74404525756836 avg_pri_loss=17.105175781249997 avg_pred_loss=0.6667458027601243


epoch: 1 train_loss=10.074198150634766 val_outcome_error=0.6658813969736632 val_reconstruction_error=5.258620689655173
  avg_predicted_sigma=0.0008100407430902123
 avg_lik_loss=2.6047301411628725 avg_pri_loss=16.90139808654785 avg_pred_loss=0.9498429119586944


epoch: 2 train_loss=10.130093288421632 val_outcome_error=0.44873170406445445 val_reconstruction_error=1.1551724137931034
  avg_predicted_sigma=0.001116796163842082
 avg_lik_loss=2.6447413682937624 avg_pri_loss=16.56599311828613 avg_pred_loss=1.8808278679847716


epoch: 3 train_loss=11.674205589294434 val_outcome_error=2.3556485056633045 val_reconstruction_error=3.3275862068965516
  avg_predicted_sigma=0.00038453732850030065
 avg_lik_loss=3.30690221786499 avg_pri_loss=18.14342041015625 avg_pred_loss=2.7004892170429224


...

epoch: 45 train_loss=11.520711326599123 val_outcome_error=2.3197611154238955 val_reconstruction_error=1.8448275862068966
  avg_predicted_sigma=0.0008834628970362246
 avg_lik_loss=3.1360464334487914 avg_pri_loss=17.17975940704346 avg_pred_loss=5.160591244697571


epoch: 46 train_loss=12.658457565307616 val_outcome_error=0.8734555515470094 val_reconstruction_error=2.4655172413793105
  avg_predicted_sigma=0.0008100976701825857
 avg_lik_loss=3.2015382051467896 avg_pri_loss=19.573324203491207 avg_pred_loss=4.781756865978242


epoch: 47 train_loss=13.483412742614746 val_outcome_error=4.421947725258598 val_reconstruction_error=2.896551724137931
  avg_predicted_sigma=0.0006131476256996393
 avg_lik_loss=3.174249029159546 avg_pri_loss=21.776716232299805 avg_pred_loss=3.7111320972442625


epoch: 48 train_loss=13.317180156707764 val_outcome_error=0.4600299877776621 val_reconstruction_error=1.396551724137931
  avg_predicted_sigma=0.001269395463168621
 avg_lik_loss=3.2029409646987914 avg_pri_loss=21.960730552673343 avg_pred_loss=2.24182323217392


epoch: 49 train_loss=12.853093338012695 val_outcome_error=0.969302248152102 val_reconstruction_error=0.7241379310344828
  avg_predicted_sigma=0.0011330745182931423
 avg_lik_loss=3.050669312477112 avg_pri_loss=21.350823020935056 avg_pred_loss=2.096155995130539


In [29]:
model.save('./output/1016-2nd045.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.50, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd045.bin


epoch: 0 train_loss=12.863917541503907 val_outcome_error=5.77585769229471 val_reconstruction_error=1.8275862068965518
  avg_predicted_sigma=0.0011012196773663163
 avg_lik_loss=2.791863036155701 avg_pri_loss=20.274580574035642 avg_pred_loss=1.28346449136734


epoch: 1 train_loss=12.526704883575439 val_outcome_error=1.2716839436911944 val_reconstruction_error=3.586206896551724
  avg_predicted_sigma=0.001099811983294785
 avg_lik_loss=2.8633613348007207 avg_pri_loss=19.292787933349608 avg_pred_loss=1.8445867538452145


epoch: 2 train_loss=11.82965030670166 val_outcome_error=0.5474726299195085 val_reconstruction_error=2.7586206896551726
  avg_predicted_sigma=0.0012056812411174178
 avg_lik_loss=2.760301327705383 avg_pri_loss=18.455976867675783 avg_pred_loss=1.0012648701667786


epoch: 3 train_loss=11.49243907928467 val_outcome_error=1.2004264105198712 val_reconstruction_error=7.344827586206897
  avg_predicted_sigma=0.0014469289453700185
 avg_lik_loss=2.787945795059204 avg_pri_loss=17.699079513549805 avg_pred_loss=1.167066180706024


...

epoch: 45 train_loss=13.457758808135987 val_outcome_error=1.8618048432578957 val_reconstruction_error=5.051724137931035
  avg_predicted_sigma=0.0002799773938022554
 avg_lik_loss=3.0801963806152344 avg_pri_loss=20.042637443542482 avg_pred_loss=3.5925106287002566


epoch: 46 train_loss=13.207446956634524 val_outcome_error=1.356872926814228 val_reconstruction_error=4.155172413793103
  avg_predicted_sigma=0.00028240514802746475
 avg_lik_loss=2.8885155558586124 avg_pri_loss=20.60030765533447 avg_pred_loss=1.722327595949173


epoch: 47 train_loss=13.000985622406004 val_outcome_error=0.9153146843448737 val_reconstruction_error=0.9655172413793104
  avg_predicted_sigma=0.00022925912344362587
 avg_lik_loss=2.873657703399658 avg_pri_loss=20.369646263122558 avg_pred_loss=1.3488062024116516


epoch: 48 train_loss=12.659322738647461 val_outcome_error=1.4906470302118908 val_reconstruction_error=1.0172413793103448
  avg_predicted_sigma=0.0003922294999938458
 avg_lik_loss=2.7991319417953493 avg_pri_loss=19.7633451461792 avg_pred_loss=1.5869468241930007


epoch: 49 train_loss=12.414992618560792 val_outcome_error=1.4643038176346335 val_reconstruction_error=2.8275862068965516
  avg_predicted_sigma=0.00036960479337722063
 avg_lik_loss=2.739933180809021 avg_pri_loss=19.143802070617674 avg_pred_loss=2.2928159296512605


In [30]:
# model.save('./output/1016-2nd045.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.50, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=11.98434829711914 val_outcome_error=1.5455702004208218 val_reconstruction_error=2.2586206896551726
  avg_predicted_sigma=0.0004703636222984642
 avg_lik_loss=2.7446126937866206 avg_pri_loss=18.66419830322266 avg_pred_loss=1.3332870990037917


epoch: 1 train_loss=11.780672931671143 val_outcome_error=0.9306902234042118 val_reconstruction_error=0.7241379310344828
  avg_predicted_sigma=0.00015427047037519515
 avg_lik_loss=2.8560984134674077 avg_pri_loss=18.12720890045166 avg_pred_loss=1.1827688485383987


epoch: 2 train_loss=11.443622684478756 val_outcome_error=0.8016271761452974 val_reconstruction_error=0.5172413793103449
  avg_predicted_sigma=4.419059041538276e-05
 avg_lik_loss=2.8791468143463135 avg_pri_loss=17.557929801940922 avg_pred_loss=0.8648445919156075


epoch: 3 train_loss=11.267432403564452 val_outcome_error=0.7414231839371559 val_reconstruction_error=0.10344827586206896
  avg_predicted_sigma=0.00039659711183048785
 avg_lik_loss=2.7531169414520265 avg_pri_loss=17.108240509033205 avg_pred_loss=1.7919674396514893


epoch: 4 train_loss=11.150635814666748 val_outcome_error=1.0016350793832194 val_reconstruction_error=1.3620689655172413
  avg_predicted_sigma=0.00041344863711856306
 avg_lik_loss=2.9073482036590574 avg_pri_loss=16.801589012145996 avg_pred_loss=1.2553921669721602


...

epoch: 45 train_loss=10.596447849273684 val_outcome_error=0.8363927015578744 val_reconstruction_error=2.9827586206896552
  avg_predicted_sigma=0.0007635360234417021
 avg_lik_loss=2.853550601005554 avg_pri_loss=15.856038379669188 avg_pred_loss=1.1983645945787431


epoch: 46 train_loss=10.254817104339601 val_outcome_error=0.32868063793752744 val_reconstruction_error=8.068965517241379
  avg_predicted_sigma=0.0007138513028621674
 avg_lik_loss=2.606881356239319 avg_pri_loss=15.68362579345703 avg_pred_loss=1.110122427344322


epoch: 47 train_loss=10.492894363403321 val_outcome_error=1.6480952596433072 val_reconstruction_error=2.103448275862069
  avg_predicted_sigma=0.0008982903091236949
 avg_lik_loss=2.8077131509780884 avg_pri_loss=15.616739177703858 avg_pred_loss=1.5370354175567629


epoch: 48 train_loss=10.305893039703369 val_outcome_error=1.0963888937899418 val_reconstruction_error=0.10344827586206896
  avg_predicted_sigma=0.0010167080909013748
 avg_lik_loss=2.558432865142822 avg_pri_loss=15.605023860931396 avg_pred_loss=1.8254007816314697


epoch: 49 train_loss=10.72922897338867 val_outcome_error=1.6275292535717127 val_reconstruction_error=0.6551724137931034
  avg_predicted_sigma=0.0015400620177388191
 avg_lik_loss=2.915965414047241 avg_pri_loss=15.678192043304442 avg_pred_loss=2.059515917301178


In [31]:
model.save('./output/1016-2nd050.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.55, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd050.bin


epoch: 0 train_loss=11.433234786987306 val_outcome_error=5.4030133346121865 val_reconstruction_error=2.0517241379310347
  avg_predicted_sigma=0.0005711045814678073
 avg_lik_loss=2.7767081975936887 avg_pri_loss=15.968954277038574 avg_pred_loss=1.7402144312858578


epoch: 1 train_loss=12.11528720855713 val_outcome_error=4.7251713470252845 val_reconstruction_error=0.5172413793103449
  avg_predicted_sigma=0.000497988483402878
 avg_lik_loss=2.9558721065521243 avg_pri_loss=16.400612640380857 avg_pred_loss=3.1381377458572386


epoch: 2 train_loss=12.095349502563478 val_outcome_error=1.6739962301703617 val_reconstruction_error=0.6551724137931034
  avg_predicted_sigma=0.0005570029024966061
 avg_lik_loss=2.881426024436951 avg_pri_loss=16.713466835021972 avg_pred_loss=2.482724118232727


epoch: 3 train_loss=11.840367031097411 val_outcome_error=0.7267644042568289 val_reconstruction_error=3.0
  avg_predicted_sigma=0.0007848557434044778
 avg_lik_loss=2.691848564147949 avg_pri_loss=16.919157218933105 avg_pred_loss=1.4896332740783693


...

epoch: 45 train_loss=11.829542064666747 val_outcome_error=2.4056989666765363 val_reconstruction_error=1.1379310344827587
  avg_predicted_sigma=0.0004511326551437378
 avg_lik_loss=2.8319704055786135 avg_pri_loss=15.923181343078614 avg_pred_loss=3.6685631394386284


epoch: 46 train_loss=13.117152881622312 val_outcome_error=3.8165024489258146 val_reconstruction_error=3.2586206896551726
  avg_predicted_sigma=0.0008240365423262119
 avg_lik_loss=3.0283058404922483 avg_pri_loss=16.918604660034177 avg_pred_loss=6.4717061519622785


epoch: 47 train_loss=13.839502048492433 val_outcome_error=2.8545190385680708 val_reconstruction_error=4.672413793103448
  avg_predicted_sigma=0.0001590004685567692
 avg_lik_loss=3.1945659160614013 avg_pri_loss=18.56898021697998 avg_pred_loss=4.557998561859131


epoch: 48 train_loss=14.267084789276122 val_outcome_error=1.4635902452730103 val_reconstruction_error=1.896551724137931
  avg_predicted_sigma=0.0001828933454817161
 avg_lik_loss=3.0726971387863156 avg_pri_loss=19.61794338226318 avg_pred_loss=4.302974784374237


epoch: 49 train_loss=13.9217604637146 val_outcome_error=1.5201339737521367 val_reconstruction_error=5.241379310344827
  avg_predicted_sigma=0.0006234224420040846
 avg_lik_loss=3.048936176300049 avg_pri_loss=19.513961982727054 avg_pred_loss=2.9294725894927973


In [32]:
# model.save('./output/1016-2nd050.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.55, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=13.986681652069091 val_outcome_error=1.3780830688005872 val_reconstruction_error=2.189655172413793
  avg_predicted_sigma=0.0004884290974587202
 avg_lik_loss=2.9265846729278566 avg_pri_loss=19.554560852050784 avg_pred_loss=3.752272617816925


epoch: 1 train_loss=14.069213199615477 val_outcome_error=3.2318219414342146 val_reconstruction_error=3.1379310344827585
  avg_predicted_sigma=0.00029547582380473614
 avg_lik_loss=3.0563132047653196 avg_pri_loss=19.911850738525388 avg_pred_loss=2.489955818653107


epoch: 2 train_loss=14.103543663024901 val_outcome_error=1.2383858387091973 val_reconstruction_error=2.0172413793103448
  avg_predicted_sigma=0.00015692083979956806
 avg_lik_loss=3.1478102922439577 avg_pri_loss=19.797788238525392 avg_pred_loss=2.5516708135604858


epoch: 3 train_loss=13.865676975250244 val_outcome_error=16.868814260075286 val_reconstruction_error=2.5
  avg_predicted_sigma=1.599690949660726e-05
 avg_lik_loss=3.0677061319351195 avg_pri_loss=19.66005516052246 avg_pred_loss=2.1159067332744597


epoch: 4 train_loss=14.671706581115723 val_outcome_error=5.709286060063041 val_reconstruction_error=1.8103448275862069
  avg_predicted_sigma=0.00043191536678932607
 avg_lik_loss=3.1503586053848265 avg_pri_loss=19.718803024291997 avg_pred_loss=5.727492022514344


...

epoch: 45 train_loss=12.855462074279783 val_outcome_error=3.3167657415930325 val_reconstruction_error=0.7241379310344828
  avg_predicted_sigma=2.3784564291418064e-06
 avg_lik_loss=2.709104084968567 avg_pri_loss=17.5631872177124 avg_pred_loss=4.801939463615417


epoch: 46 train_loss=12.90192584991455 val_outcome_error=4.19452346966207 val_reconstruction_error=3.1206896551724137
  avg_predicted_sigma=1.765013439580798e-05
 avg_lik_loss=2.826352047920227 avg_pri_loss=18.067957305908205 avg_pred_loss=2.980680084228515


epoch: 47 train_loss=13.876483058929441 val_outcome_error=1.802809978906394 val_reconstruction_error=0.7586206896551724
  avg_predicted_sigma=9.45970714383293e-06
 avg_lik_loss=3.0822215795516965 avg_pri_loss=19.280500984191892 avg_pred_loss=3.2203488528728483


epoch: 48 train_loss=14.00512924194336 val_outcome_error=1.0395254941165089 val_reconstruction_error=0.5172413793103449
  avg_predicted_sigma=1.1023867045878433e-05
 avg_lik_loss=2.921116590499878 avg_pri_loss=20.276789093017584 avg_pred_loss=1.756762847304344


epoch: 49 train_loss=14.508083724975585 val_outcome_error=0.7127595902002143 val_reconstruction_error=0.46551724137931033
  avg_predicted_sigma=0.00010573499457677826
 avg_lik_loss=3.269639682769775 avg_pri_loss=20.531287002563477 avg_pred_loss=1.9006968021392823


In [33]:
model.save('./output/1016-2nd055.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.60, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd055.bin


epoch: 0 train_loss=14.342677307128904 val_outcome_error=0.9952106563888005 val_reconstruction_error=2.0517241379310347
  avg_predicted_sigma=1.4999861377873458e-05
 avg_lik_loss=2.7508435964584352 avg_pri_loss=19.86028232574463 avg_pred_loss=0.829808296263218


epoch: 1 train_loss=13.952747631072999 val_outcome_error=1.040822964047751 val_reconstruction_error=2.586206896551724
  avg_predicted_sigma=0.00044033501762896776
 avg_lik_loss=2.699533033370972 avg_pri_loss=19.104597282409667 avg_pred_loss=1.4449513792991637


epoch: 2 train_loss=13.319148635864257 val_outcome_error=0.5866694026594352 val_reconstruction_error=0.29310344827586204
  avg_predicted_sigma=0.00018696588813327253
 avg_lik_loss=2.6370013236999514 avg_pri_loss=18.29454650878906 avg_pred_loss=1.0196761846542357


epoch: 3 train_loss=12.948594760894775 val_outcome_error=0.5894457308648761 val_reconstruction_error=1.0689655172413792
  avg_predicted_sigma=0.0003448343195486814
 avg_lik_loss=2.7927428960800165 avg_pri_loss=17.562694358825684 avg_pred_loss=0.6370766878128051


...

epoch: 45 train_loss=11.605938720703124 val_outcome_error=1.1464379240276343 val_reconstruction_error=1.1379310344827587
  avg_predicted_sigma=0.00029444010579027236
 avg_lik_loss=2.643086051940918 avg_pri_loss=15.492427921295164 avg_pred_loss=0.9431602209806442


epoch: 46 train_loss=11.324807357788085 val_outcome_error=4.818089005466607 val_reconstruction_error=0.41379310344827586
  avg_predicted_sigma=0.00031228980515152216
 avg_lik_loss=2.5881305932998657 avg_pri_loss=15.20608568191528 avg_pred_loss=0.6560643911361694


epoch: 47 train_loss=11.513654232025145 val_outcome_error=0.5147107284861211 val_reconstruction_error=0.0
  avg_predicted_sigma=0.00022913195425644517
 avg_lik_loss=2.7283683776855465 avg_pri_loss=15.047667598724363 avg_pred_loss=1.4427078604698185


epoch: 48 train_loss=11.459594631195069 val_outcome_error=1.0041371117526126 val_reconstruction_error=0.0
  avg_predicted_sigma=0.00023018252977635711
 avg_lik_loss=2.775295114517212 avg_pri_loss=15.15795564651489 avg_pred_loss=0.5841350689530374


epoch: 49 train_loss=11.528003787994384 val_outcome_error=0.8764057633138026 val_reconstruction_error=0.4482758620689655
  avg_predicted_sigma=0.0006825317395851016
 avg_lik_loss=2.732860767841339 avg_pri_loss=15.208728885650634 avg_pred_loss=0.9867879211902619


In [34]:
model.save('./output/1016-2nd060.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.65, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd060.bin


epoch: 0 train_loss=12.288943195343018 val_outcome_error=0.4967428389692048 val_reconstruction_error=2.2413793103448274
  avg_predicted_sigma=0.0002078768884530291
 avg_lik_loss=2.6164847493171695 avg_pri_loss=15.395302867889404 avg_pred_loss=1.1755960792303086


epoch: 1 train_loss=12.692416095733641 val_outcome_error=0.5443839427528006 val_reconstruction_error=6.741379310344827
  avg_predicted_sigma=0.0005263218190521002
 avg_lik_loss=3.0586891412734984 avg_pri_loss=15.511079120635987 avg_pred_loss=0.6994799882173539


epoch: 2 train_loss=12.649168872833252 val_outcome_error=0.29932617292267427 val_reconstruction_error=0.5172413793103449
  avg_predicted_sigma=3.7045836506877095e-05
 avg_lik_loss=3.0310659885406492 avg_pri_loss=15.454616355895995 avg_pred_loss=0.8040369898080827


epoch: 3 train_loss=12.36311140060425 val_outcome_error=0.4816558599097876 val_reconstruction_error=0.5517241379310345
  avg_predicted_sigma=0.00024226363166235387
 avg_lik_loss=2.8849972963333133 avg_pri_loss=15.300531482696533 avg_pred_loss=0.5618582747876644


...

epoch: 45 train_loss=11.695360946655274 val_outcome_error=0.1670521038444309 val_reconstruction_error=2.5517241379310347
  avg_predicted_sigma=0.001036634435877204
 avg_lik_loss=2.455652475357055 avg_pri_loss=14.873455905914305 avg_pred_loss=0.6504356473684313


epoch: 46 train_loss=11.546891403198243 val_outcome_error=0.3359884360088547 val_reconstruction_error=1.0689655172413792
  avg_predicted_sigma=0.0009422583389095962
 avg_lik_loss=2.4562528371810917 avg_pri_loss=14.716836738586426 avg_pred_loss=0.4039524927735329


epoch: 47 train_loss=11.441561317443847 val_outcome_error=0.1850910306159251 val_reconstruction_error=0.9827586206896551
  avg_predicted_sigma=0.0008556998800486326
 avg_lik_loss=2.428245413303375 avg_pri_loss=14.598479175567627 avg_pred_loss=0.39315142668783665


epoch: 48 train_loss=11.591831302642824 val_outcome_error=1.3714567601262908 val_reconstruction_error=0.3448275862068966
  avg_predicted_sigma=0.001416871091350913
 avg_lik_loss=2.599037837982178 avg_pri_loss=14.52664051055908 avg_pred_loss=0.5723982200026512


epoch: 49 train_loss=11.606740760803222 val_outcome_error=0.8529137216778344 val_reconstruction_error=0.22413793103448276
  avg_predicted_sigma=0.001476980047300458
 avg_lik_loss=2.415737521648407 avg_pri_loss=14.607387828826901 avg_pred_loss=1.2810038402676582


In [35]:
model.save('./output/1016-2nd065.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.70, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd065.bin


epoch: 0 train_loss=12.259935283660889 val_outcome_error=1.2037628150800812 val_reconstruction_error=0.6379310344827587
  avg_predicted_sigma=0.001410339493304491
 avg_lik_loss=2.3828750133514407 avg_pri_loss=14.665004444122314 avg_pred_loss=1.0572962254285811


epoch: 1 train_loss=12.562960338592529 val_outcome_error=0.27175437830306726 val_reconstruction_error=0.0
  avg_predicted_sigma=0.0011668495135381818
 avg_lik_loss=2.729346513748169 avg_pri_loss=14.708697605133056 avg_pred_loss=0.7627853035926818


epoch: 2 train_loss=12.5394229888916 val_outcome_error=1.172136746676269 val_reconstruction_error=0.6724137931034483
  avg_predicted_sigma=0.0013146670535206795
 avg_lik_loss=2.530191946029663 avg_pri_loss=14.692856216430663 avg_pred_loss=1.6807274252176285


epoch: 3 train_loss=12.679078769683839 val_outcome_error=0.5897421337209363 val_reconstruction_error=1.293103448275862
  avg_predicted_sigma=0.0013610700843855739
 avg_lik_loss=2.7248807311058045 avg_pri_loss=14.794393348693848 avg_pred_loss=1.0780554816126824


...

epoch: 45 train_loss=12.268101310729982 val_outcome_error=0.3502327744976448 val_reconstruction_error=0.6551724137931034
  avg_predicted_sigma=0.0029786478262394667
 avg_lik_loss=2.6265129566192624 avg_pri_loss=14.450908946990966 avg_pred_loss=0.6545052796602249


epoch: 46 train_loss=12.48109359741211 val_outcome_error=0.46484626222535463 val_reconstruction_error=0.0
  avg_predicted_sigma=0.002749610925093293
 avg_lik_loss=2.909231996536255 avg_pri_loss=14.377919864654542 avg_pred_loss=0.629306560754776


epoch: 47 train_loss=12.28314666748047 val_outcome_error=0.9744064476988268 val_reconstruction_error=1.1206896551724137
  avg_predicted_sigma=0.003219903213903308
 avg_lik_loss=2.715025854110718 avg_pri_loss=14.355721569061277 avg_pred_loss=0.6393768697977066


epoch: 48 train_loss=12.269023227691651 val_outcome_error=1.3445678400904761 val_reconstruction_error=2.0689655172413794
  avg_predicted_sigma=0.0019842933397740126
 avg_lik_loss=2.6540056228637696 avg_pri_loss=14.35790958404541 avg_pred_loss=0.8603587046265601


epoch: 49 train_loss=12.068124771118164 val_outcome_error=0.6715447314653772 val_reconstruction_error=1.5
  avg_predicted_sigma=0.0027379377279430628
 avg_lik_loss=2.473536622524262 avg_pri_loss=14.34354705810547 avg_pred_loss=0.759135389328003


In [36]:
model.save('./output/1016-2nd070.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.75, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd070.bin


epoch: 0 train_loss=12.860644435882566 val_outcome_error=0.4233211905761654 val_reconstruction_error=1.9310344827586208
  avg_predicted_sigma=0.003043636679649353
 avg_lik_loss=2.6113227844238285 avg_pri_loss=14.38998718261719 avg_pred_loss=0.5082937777042388


epoch: 1 train_loss=12.83575210571289 val_outcome_error=1.5680672532896283 val_reconstruction_error=0.1206896551724138
  avg_predicted_sigma=0.0024978271685540676
 avg_lik_loss=2.528366541862488 avg_pri_loss=14.397458648681642 avg_pred_loss=0.7619145333766938


epoch: 2 train_loss=13.22471990585327 val_outcome_error=2.063589996451083 val_reconstruction_error=0.4827586206896552
  avg_predicted_sigma=0.0026442070957273245
 avg_lik_loss=2.850238490104675 avg_pri_loss=14.399314498901367 avg_pred_loss=1.1881691694259642


epoch: 3 train_loss=13.568466091156004 val_outcome_error=0.9234539292663739 val_reconstruction_error=0.5862068965517241
  avg_predicted_sigma=0.003018640447407961
 avg_lik_loss=3.0028936147689818 avg_pri_loss=14.536404895782471 avg_pred_loss=1.6960438072681425


...

epoch: 45 train_loss=15.404988765716551 val_outcome_error=1.7565422605579997 val_reconstruction_error=0.8103448275862069
  avg_predicted_sigma=0.0013979757204651833
 avg_lik_loss=2.5910149097442625 avg_pri_loss=17.338919258117674 avg_pred_loss=2.5572061419487


epoch: 46 train_loss=14.94177312850952 val_outcome_error=1.4788912574483282 val_reconstruction_error=2.2241379310344827
  avg_predicted_sigma=0.0014890468446537852
 avg_lik_loss=2.7534049034118655 avg_pri_loss=16.883903121948247 avg_pred_loss=1.0885430425405505


epoch: 47 train_loss=14.944243717193602 val_outcome_error=1.6724127233154698 val_reconstruction_error=0.0
  avg_predicted_sigma=0.0018312858883291483
 avg_lik_loss=2.6496830463409426 avg_pri_loss=16.47705020904541 avg_pred_loss=3.170922166109085


epoch: 48 train_loss=14.996070384979248 val_outcome_error=8.11472197741034 val_reconstruction_error=0.9482758620689655
  avg_predicted_sigma=0.0017840784275904298
 avg_lik_loss=2.870991730690002 avg_pri_loss=16.09578437805176 avg_pred_loss=3.8067806720733643


epoch: 49 train_loss=14.714385128021238 val_outcome_error=6.372431508083 val_reconstruction_error=0.7758620689655172
  avg_predicted_sigma=0.001319597358815372
 avg_lik_loss=2.7684373617172238 avg_pri_loss=15.962675189971923 avg_pred_loss=3.3586780071258544


In [37]:
# model.save('./output/1016-2nd070.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.75, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=14.840341663360595 val_outcome_error=8.304720042609874 val_reconstruction_error=1.3103448275862069
  avg_predicted_sigma=0.0014517243253067136
 avg_lik_loss=2.8512050628662107 avg_pri_loss=16.19156541824341 avg_pred_loss=2.7289222240448


epoch: 1 train_loss=15.259873676300048 val_outcome_error=1.521033945703412 val_reconstruction_error=0.6551724137931034
  avg_predicted_sigma=0.0007812929106876254
 avg_lik_loss=3.074516201019287 avg_pri_loss=16.335599899291992 avg_pred_loss=3.2559490680694583


epoch: 2 train_loss=14.888147354125975 val_outcome_error=1.417697783038188 val_reconstruction_error=0.7413793103448276
  avg_predicted_sigma=0.0006784199504181743
 avg_lik_loss=2.9449040174484256 avg_pri_loss=16.277750968933105 avg_pred_loss=2.1850089609622954


epoch: 3 train_loss=14.932777595520019 val_outcome_error=3.0269483394833996 val_reconstruction_error=4.810344827586207
  avg_predicted_sigma=0.0006442628218792379
 avg_lik_loss=3.0533808946609495 avg_pri_loss=16.11743631362915 avg_pred_loss=2.4955840289592746


epoch: 4 train_loss=14.394916343688966 val_outcome_error=0.9552910049954886 val_reconstruction_error=0.3448275862068966
  avg_predicted_sigma=0.0012526295613497496
 avg_lik_loss=2.7605202674865725 avg_pri_loss=16.00794038772583 avg_pred_loss=1.5634434401988981


...

epoch: 45 train_loss=12.995336055755615 val_outcome_error=0.5033382372938427 val_reconstruction_error=0.0
  avg_predicted_sigma=0.0014513408532366157
 avg_lik_loss=2.6677133083343505 avg_pri_loss=14.422583103179932 avg_pred_loss=0.8170452356338501


epoch: 46 train_loss=12.768379974365237 val_outcome_error=0.35001353996843776 val_reconstruction_error=0.41379310344827586
  avg_predicted_sigma=0.0006970141548663378
 avg_lik_loss=2.467163467407226 avg_pri_loss=14.396354293823244 avg_pred_loss=0.7279777020215988


epoch: 47 train_loss=12.867024326324461 val_outcome_error=0.2564750578117606 val_reconstruction_error=1.1724137931034482
  avg_predicted_sigma=0.0011108452454209328
 avg_lik_loss=2.6107277870178223 avg_pri_loss=14.390112781524657 avg_pred_loss=0.5561204396188258


epoch: 48 train_loss=12.86795253753662 val_outcome_error=1.548286845123577 val_reconstruction_error=0.0
  avg_predicted_sigma=0.0012021752772852778
 avg_lik_loss=2.5782200694084167 avg_pri_loss=14.34669198989868 avg_pred_loss=0.8872492924332618


epoch: 49 train_loss=12.817894172668456 val_outcome_error=0.4538974786789101 val_reconstruction_error=0.25862068965517243
  avg_predicted_sigma=0.0006792026106268167
 avg_lik_loss=2.4805540800094605 avg_pri_loss=14.330146408081054 avg_pred_loss=1.1730323776602745


In [38]:
model.save('./output/1016-2nd075.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.80, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd075.bin


epoch: 0 train_loss=13.519296741485597 val_outcome_error=3.499962836275976 val_reconstruction_error=0.6379310344827587
  avg_predicted_sigma=0.0012546152574941516
 avg_lik_loss=2.4950333833694454 avg_pri_loss=14.318094730377195 avg_pred_loss=1.2847543448209764


epoch: 1 train_loss=13.789704608917239 val_outcome_error=1.7666623772946843 val_reconstruction_error=1.4827586206896552
  avg_predicted_sigma=0.0005181733286008239
 avg_lik_loss=2.738183832168579 avg_pri_loss=14.34814281463623 avg_pred_loss=1.3677021116018293


epoch: 2 train_loss=13.639582061767578 val_outcome_error=1.626799911399497 val_reconstruction_error=1.0172413793103448
  avg_predicted_sigma=0.00118157803080976
 avg_lik_loss=2.6933589220046996 avg_pri_loss=14.363512229919436 avg_pred_loss=0.7472012430429459


epoch: 3 train_loss=13.35704526901245 val_outcome_error=1.1286727226705682 val_reconstruction_error=0.9310344827586207
  avg_predicted_sigma=0.0012632765574380755
 avg_lik_loss=2.455381941795349 avg_pri_loss=14.311567211151123 avg_pred_loss=0.6636571377515793


...

epoch: 45 train_loss=14.367269515991211 val_outcome_error=0.5337943536308286 val_reconstruction_error=0.7931034482758621
  avg_predicted_sigma=0.0009801601991057396
 avg_lik_loss=2.687809586524964 avg_pri_loss=15.335154819488526 avg_pred_loss=0.6244985058903695


epoch: 46 train_loss=14.24370355606079 val_outcome_error=0.30008433470543383 val_reconstruction_error=2.9655172413793105
  avg_predicted_sigma=0.0005611969972960651
 avg_lik_loss=2.7243792772293087 avg_pri_loss=15.082341384887693 avg_pred_loss=0.8252111047506333


epoch: 47 train_loss=14.02131175994873 val_outcome_error=0.33933979711868223 val_reconstruction_error=0.46551724137931033
  avg_predicted_sigma=0.00077608204446733
 avg_lik_loss=2.586576867103576 avg_pri_loss=14.817294025421141 avg_pred_loss=1.4231318324804305


epoch: 48 train_loss=13.922492599487304 val_outcome_error=0.5097449543085499 val_reconstruction_error=0.6551724137931034
  avg_predicted_sigma=0.0010775686241686344
 avg_lik_loss=2.7442184686660767 avg_pri_loss=14.603980922698977 avg_pred_loss=0.9943404823541642


epoch: 49 train_loss=13.867240715026856 val_outcome_error=0.535716232186821 val_reconstruction_error=0.5172413793103449
  avg_predicted_sigma=0.0011894578346982598
 avg_lik_loss=2.6256495714187627 avg_pri_loss=14.490750885009763 avg_pred_loss=1.7508682966232303


In [39]:
model.save('./output/1016-2nd080.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.85, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd080.bin


epoch: 0 train_loss=14.632570838928222 val_outcome_error=0.33026100965749366 val_reconstruction_error=1.293103448275862
  avg_predicted_sigma=0.0010178287047892809
 avg_lik_loss=2.7525130510330205 avg_pri_loss=14.565411853790282 avg_pred_loss=1.234036695957184


epoch: 1 train_loss=14.511363983154299 val_outcome_error=1.4025546308731895 val_reconstruction_error=2.6206896551724137
  avg_predicted_sigma=0.0013823643093928695
 avg_lik_loss=2.6229448080062863 avg_pri_loss=14.616118526458738 avg_pred_loss=1.0269357889890671


epoch: 2 train_loss=14.865076446533203 val_outcome_error=11.632205928341167 val_reconstruction_error=0.7758620689655172
  avg_predicted_sigma=0.0006971104303374887
 avg_lik_loss=2.8132753133773805 avg_pri_loss=14.646349525451662 avg_pred_loss=1.7976500332355496


epoch: 3 train_loss=15.883745574951169 val_outcome_error=12.129470674719228 val_reconstruction_error=1.1379310344827587
  avg_predicted_sigma=0.0002569527132436633
 avg_lik_loss=2.985273337364197 avg_pri_loss=14.910250759124754 avg_pred_loss=5.117665529251098


...

epoch: 45 train_loss=15.125962448120118 val_outcome_error=0.27113619924017246 val_reconstruction_error=0.5172413793103449
  avg_predicted_sigma=0.0007828323869034648
 avg_lik_loss=2.793118262290955 avg_pri_loss=15.316554260253906 avg_pred_loss=0.3928038232028484


epoch: 46 train_loss=14.759615230560303 val_outcome_error=0.18212452951597216 val_reconstruction_error=1.3793103448275863
  avg_predicted_sigma=0.00019173926557414234
 avg_lik_loss=2.671488976478576 avg_pri_loss=15.012254142761229 avg_pred_loss=0.3889251485466957


epoch: 47 train_loss=14.414455127716062 val_outcome_error=1.5545145767748512 val_reconstruction_error=2.0172413793103448
  avg_predicted_sigma=0.0007705792086198926
 avg_lik_loss=2.463645839691162 avg_pri_loss=14.754707813262938 avg_pred_loss=0.7214524447917939


epoch: 48 train_loss=14.344907760620119 val_outcome_error=1.0213247874801406 val_reconstruction_error=2.1206896551724137
  avg_predicted_sigma=0.0005951985367573798
 avg_lik_loss=2.5181560277938844 avg_pri_loss=14.60667791366577 avg_pred_loss=0.7230256855487822


epoch: 49 train_loss=14.628701496124265 val_outcome_error=1.2022866727540087 val_reconstruction_error=5.913793103448276
  avg_predicted_sigma=0.0008242928888648748
 avg_lik_loss=2.6445510983467098 avg_pri_loss=14.664033985137937 avg_pred_loss=1.3291850507259366


In [40]:
# model.save('./output/1016-2nd080.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.85, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=15.190894699096676 val_outcome_error=2.766397584873925 val_reconstruction_error=1.0344827586206897
  avg_predicted_sigma=0.0005924393190070987
 avg_lik_loss=2.5619069814682005 avg_pri_loss=14.854819393157957 avg_pred_loss=3.845640587806701


epoch: 1 train_loss=15.728016376495361 val_outcome_error=4.332326464909953 val_reconstruction_error=0.3620689655172414
  avg_predicted_sigma=2.791701626847498e-05
 avg_lik_loss=2.924812746047974 avg_pri_loss=15.483008289337157 avg_pred_loss=2.1618532836437225


epoch: 2 train_loss=16.161555004119872 val_outcome_error=0.4672633030860858 val_reconstruction_error=1.7241379310344827
  avg_predicted_sigma=0.0008492913329973817
 avg_lik_loss=2.8637182235717775 avg_pri_loss=16.062640380859374 avg_pred_loss=2.2411800831556317


epoch: 3 train_loss=15.8767520904541 val_outcome_error=0.19898741125170263 val_reconstruction_error=2.5517241379310347
  avg_predicted_sigma=0.0007117684581317008
 avg_lik_loss=2.8263654947280887 avg_pri_loss=16.08916244506836 avg_pred_loss=0.8313005968928336


epoch: 4 train_loss=15.399067306518553 val_outcome_error=0.2943066900920929 val_reconstruction_error=1.9655172413793103
  avg_predicted_sigma=0.00014794715389143676
 avg_lik_loss=2.5747098684310914 avg_pri_loss=15.798688316345215 avg_pred_loss=0.8319322988390924


...

epoch: 45 train_loss=14.066781997680664 val_outcome_error=1.7662618660153158 val_reconstruction_error=0.7931034482758621
  avg_predicted_sigma=0.0016640941612422466
 avg_lik_loss=2.4455140352249147 avg_pri_loss=14.342163181304931 avg_pred_loss=0.763135325908661


epoch: 46 train_loss=14.111901473999026 val_outcome_error=0.47122244697025417 val_reconstruction_error=0.5
  avg_predicted_sigma=0.0007438685861416161
 avg_lik_loss=2.4340167760849 avg_pri_loss=14.357455921173093 avg_pred_loss=0.9891522228717804


epoch: 47 train_loss=14.010786914825438 val_outcome_error=0.3831415557700094 val_reconstruction_error=0.41379310344827586
  avg_predicted_sigma=0.0012699859216809273
 avg_lik_loss=2.399409520626068 avg_pri_loss=14.338513851165771 avg_pred_loss=0.7151234656572343


epoch: 48 train_loss=14.071719932556153 val_outcome_error=0.30892911264053535 val_reconstruction_error=0.22413793103448276
  avg_predicted_sigma=0.0017275039572268724
 avg_lik_loss=2.5356993436813355 avg_pri_loss=14.311720085144044 avg_pred_loss=0.4728997200727462


epoch: 49 train_loss=13.894171619415282 val_outcome_error=0.2671107635210088 val_reconstruction_error=0.6551724137931034
  avg_predicted_sigma=0.0013002943014726043
 avg_lik_loss=2.3286531925201417 avg_pri_loss=14.2988844871521 avg_pred_loss=0.6274488732218743


In [41]:
model.save('./output/1016-2nd085.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.90, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd085.bin


epoch: 0 train_loss=14.549703121185303 val_outcome_error=0.7652891339756257 val_reconstruction_error=1.0172413793103448
  avg_predicted_sigma=0.0017884384142234921
 avg_lik_loss=2.3152771472930906 avg_pri_loss=14.255582523345948 avg_pred_loss=0.7910227477550507


epoch: 1 train_loss=14.842125797271727 val_outcome_error=0.5075914222681995 val_reconstruction_error=1.4310344827586208
  avg_predicted_sigma=0.0011815644102171063
 avg_lik_loss=2.5457215785980227 avg_pri_loss=14.277047061920166 avg_pred_loss=1.0766065835952758


epoch: 2 train_loss=14.710601139068604 val_outcome_error=0.236688486277375 val_reconstruction_error=0.29310344827586204
  avg_predicted_sigma=0.0021873193327337503
 avg_lik_loss=2.4720282793045047 avg_pri_loss=14.34942684173584 avg_pred_loss=0.43135165423154825


epoch: 3 train_loss=14.804020118713376 val_outcome_error=0.24913346076732937 val_reconstruction_error=1.793103448275862
  avg_predicted_sigma=0.0010486352257430553
 avg_lik_loss=2.593174958229065 avg_pri_loss=14.340686321258545 avg_pred_loss=0.35788657963275905


...

epoch: 45 train_loss=20.39891185760498 val_outcome_error=2.86854304865954 val_reconstruction_error=5.793103448275862
  avg_predicted_sigma=0.0017295826692134142
 avg_lik_loss=3.2630167245864867 avg_pri_loss=18.84254207611084 avg_pred_loss=5.902351808547975


epoch: 46 train_loss=20.2200984954834 val_outcome_error=3.3815285135765722 val_reconstruction_error=5.879310344827586
  avg_predicted_sigma=0.0015228200936689973
 avg_lik_loss=2.9055143356323243 avg_pri_loss=19.60485973358154 avg_pred_loss=3.3117489695549014


epoch: 47 train_loss=20.33548355102539 val_outcome_error=1.2293027158078738 val_reconstruction_error=2.1724137931034484
  avg_predicted_sigma=0.0014124189037829638
 avg_lik_loss=2.900765943527222 avg_pri_loss=19.588559341430663 avg_pred_loss=4.007860136032106


epoch: 48 train_loss=21.51338882446289 val_outcome_error=1.647937946849478 val_reconstruction_error=10.724137931034482
  avg_predicted_sigma=0.0011298139579594135
 avg_lik_loss=3.351631116867065 avg_pri_loss=19.62788619995117 avg_pred_loss=7.730197644233703


epoch: 49 train_loss=23.700201988220215 val_outcome_error=6.887325539709422 val_reconstruction_error=5.310344827586207
  avg_predicted_sigma=0.0012254591565579176
 avg_lik_loss=3.8428472518920898 avg_pri_loss=21.12455425262451 avg_pred_loss=9.946261167526245


In [42]:
# model.save('./output/1016-2nd085.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.90, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=24.839849853515627 val_outcome_error=4.261786991356198 val_reconstruction_error=3.3620689655172415
  avg_predicted_sigma=0.0013729293132200837
 avg_lik_loss=3.6171504735946653 avg_pri_loss=23.395906448364258 avg_pred_loss=6.776168274879455


epoch: 1 train_loss=24.719601440429688 val_outcome_error=2.1497337440265407 val_reconstruction_error=0.9482758620689655
  avg_predicted_sigma=0.0017827381379902363
 avg_lik_loss=3.4042451381683354 avg_pri_loss=24.04014663696289 avg_pred_loss=4.306197905540466


epoch: 2 train_loss=25.012915039062506 val_outcome_error=1.5124316992687836 val_reconstruction_error=1.103448275862069
  avg_predicted_sigma=0.0021766959689557552
 avg_lik_loss=3.5892368078231818 avg_pri_loss=23.97086067199707 avg_pred_loss=5.2283262491226195


epoch: 3 train_loss=23.657426834106445 val_outcome_error=1.3064999756366533 val_reconstruction_error=2.7413793103448274
  avg_predicted_sigma=0.002028676914051175
 avg_lik_loss=3.03612334728241 avg_pri_loss=23.295947265625 avg_pred_loss=3.9462902307510372


epoch: 4 train_loss=22.337204551696775 val_outcome_error=1.1248339040317865 val_reconstruction_error=1.8103448275862069
  avg_predicted_sigma=0.002607803326100111
 avg_lik_loss=2.654563927650451 avg_pri_loss=22.067358207702643 avg_pred_loss=4.4894005298614506


...

epoch: 45 train_loss=16.41091470718384 val_outcome_error=1.1861474611507228 val_reconstruction_error=2.0344827586206895
  avg_predicted_sigma=0.0013680168194696307
 avg_lik_loss=2.613265490531922 avg_pri_loss=15.733872032165523 avg_pred_loss=2.350086975097656


epoch: 46 train_loss=16.12131977081299 val_outcome_error=1.3055497916507945 val_reconstruction_error=0.3275862068965517
  avg_predicted_sigma=0.002458875998854637
 avg_lik_loss=2.3929326891899105 avg_pri_loss=15.658154296875 avg_pred_loss=2.2730237245559692


epoch: 47 train_loss=16.39106168746948 val_outcome_error=1.7952284018340705 val_reconstruction_error=2.8275862068965516
  avg_predicted_sigma=0.0013671183260157704
 avg_lik_loss=2.6203101873397827 avg_pri_loss=15.753237915039062 avg_pred_loss=2.124330914020538


epoch: 48 train_loss=16.565610980987547 val_outcome_error=8.403572720692818 val_reconstruction_error=0.0
  avg_predicted_sigma=0.0018766646971926093
 avg_lik_loss=2.652918815612793 avg_pri_loss=15.86130533218384 avg_pred_loss=2.385789525508881


epoch: 49 train_loss=17.83754253387451 val_outcome_error=4.732327117492586 val_reconstruction_error=5.155172413793103
  avg_predicted_sigma=0.0014745924854651093
 avg_lik_loss=2.642378401756286 avg_pri_loss=16.158493041992188 avg_pred_loss=7.7157101869583125


In [43]:
# model.save('./output/1016-2nd085.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.90, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=18.71055335998535 val_outcome_error=3.6503285547877025 val_reconstruction_error=1.8448275862068966
  avg_predicted_sigma=0.0012017335975542665
 avg_lik_loss=2.921122169494629 avg_pri_loss=17.103658676147457 avg_pred_loss=6.629900598526001


epoch: 1 train_loss=19.082522201538087 val_outcome_error=1.3317958498277254 val_reconstruction_error=5.120689655172414
  avg_predicted_sigma=0.001257382333278656
 avg_lik_loss=2.8315554618835446 avg_pri_loss=18.11688995361328 avg_pred_loss=4.452930545806885


epoch: 2 train_loss=19.23789825439453 val_outcome_error=2.193370578343728 val_reconstruction_error=2.1206896551724137
  avg_predicted_sigma=0.000802853493951261
 avg_lik_loss=3.122563934326172 avg_pri_loss=18.373484420776368 avg_pred_loss=2.669613575935364


epoch: 3 train_loss=18.817136192321776 val_outcome_error=1.1810424063506637 val_reconstruction_error=0.10344827586206896
  avg_predicted_sigma=0.0015332598704844713
 avg_lik_loss=2.996064472198486 avg_pri_loss=18.085323905944826 avg_pred_loss=2.402216064929962


epoch: 4 train_loss=18.633808517456053 val_outcome_error=0.6990119519141634 val_reconstruction_error=2.0172413793103448
  avg_predicted_sigma=0.0009725619456730783
 avg_lik_loss=3.3578145742416385 avg_pri_loss=17.70510787963867 avg_pred_loss=1.370595145225525


...

epoch: 45 train_loss=15.632984447479249 val_outcome_error=0.542725100167877 val_reconstruction_error=1.5172413793103448
  avg_predicted_sigma=0.0024494559038430452
 avg_lik_loss=2.8357431650161744 avg_pri_loss=14.590792179107664 avg_pred_loss=2.343993020057678


epoch: 46 train_loss=15.375067901611327 val_outcome_error=1.4021593066488576 val_reconstruction_error=2.4827586206896552
  avg_predicted_sigma=0.0017380050849169493
 avg_lik_loss=2.48350682258606 avg_pri_loss=14.737525463104248 avg_pred_loss=2.082991307973862


epoch: 47 train_loss=15.995734691619873 val_outcome_error=3.6300010576733768 val_reconstruction_error=3.6379310344827585
  avg_predicted_sigma=0.002624576911330223
 avg_lik_loss=2.8102289915084837 avg_pri_loss=14.915813922882078 avg_pred_loss=2.8944808006286618


epoch: 48 train_loss=16.187609863281246 val_outcome_error=1.2509527127551252 val_reconstruction_error=0.13793103448275862
  avg_predicted_sigma=0.002037735190242529
 avg_lik_loss=2.7808525085449216 avg_pri_loss=15.362631702423096 avg_pred_loss=2.0287885069847107


epoch: 49 train_loss=16.93529624938965 val_outcome_error=0.874545888949891 val_reconstruction_error=2.310344827586207
  avg_predicted_sigma=0.002583973342552781
 avg_lik_loss=3.1412847518920897 avg_pri_loss=15.912861824035645 avg_pred_loss=1.6628011286258697


In [44]:
model.save('./output/1016-2nd090.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=0.95, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd090.bin


epoch: 0 train_loss=17.757200813293455 val_outcome_error=0.32107007615368494 val_reconstruction_error=3.0344827586206895
  avg_predicted_sigma=0.0030325455591082573
 avg_lik_loss=2.837152242660522 avg_pri_loss=16.321438789367676 avg_pred_loss=1.6480578064918519


epoch: 1 train_loss=17.399900436401367 val_outcome_error=0.16112077393468024 val_reconstruction_error=1.9827586206896552
  avg_predicted_sigma=0.002003943547606468
 avg_lik_loss=2.7583711862564093 avg_pri_loss=16.22266674041748 avg_pred_loss=0.6452446199953555


epoch: 2 train_loss=17.3709716796875 val_outcome_error=0.817985970326732 val_reconstruction_error=1.5517241379310345
  avg_predicted_sigma=0.0026216134428977966
 avg_lik_loss=2.609594988822937 avg_pri_loss=15.93203182220459 avg_pred_loss=2.600969752669335


epoch: 3 train_loss=17.380059814453126 val_outcome_error=0.6455487153641264 val_reconstruction_error=4.0
  avg_predicted_sigma=0.0019774693064391613
 avg_lik_loss=2.9543353557586673 avg_pri_loss=15.850877761840819 avg_pred_loss=1.328016811609268


...

epoch: 45 train_loss=16.358552837371825 val_outcome_error=0.5100020805624504 val_reconstruction_error=1.4827586206896552
  avg_predicted_sigma=0.0032490920275449753
 avg_lik_loss=2.4185849189758297 avg_pri_loss=15.082567977905272 avg_pred_loss=2.287261700630188


epoch: 46 train_loss=16.55379009246826 val_outcome_error=0.4826681019099013 val_reconstruction_error=2.9482758620689653
  avg_predicted_sigma=0.003449053503572941
 avg_lik_loss=2.466366672515869 avg_pri_loss=15.37870121002197 avg_pred_loss=1.66939497590065


epoch: 47 train_loss=16.687371063232423 val_outcome_error=1.321582716096543 val_reconstruction_error=2.810344827586207
  avg_predicted_sigma=0.00224722595885396
 avg_lik_loss=2.699908781051635 avg_pri_loss=15.367294120788573 avg_pred_loss=1.2641585826873778


epoch: 48 train_loss=16.480033683776856 val_outcome_error=0.5230530273940748 val_reconstruction_error=0.5172413793103449
  avg_predicted_sigma=0.0025950849521905184
 avg_lik_loss=2.6474458336830136 avg_pri_loss=15.156760883331296 avg_pred_loss=1.4384511291980744


epoch: 49 train_loss=16.146840858459473 val_outcome_error=0.3653193682921146 val_reconstruction_error=3.396551724137931
  avg_predicted_sigma=0.0022912637796252966
 avg_lik_loss=2.663842749595642 avg_pri_loss=14.92365837097168 avg_pred_loss=0.724534259736538


In [45]:
model.save('./output/1016-2nd095.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.0,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd095.bin


epoch: 0 train_loss=16.591864967346197 val_outcome_error=0.3265166402019447 val_reconstruction_error=0.5517241379310345
  avg_predicted_sigma=0.0026467647403478622
 avg_lik_loss=2.6266835451126096 avg_pri_loss=14.705156421661377 avg_pred_loss=0.6584066569805146


epoch: 1 train_loss=16.508831405639647 val_outcome_error=0.8900003134755253 val_reconstruction_error=2.8448275862068964
  avg_predicted_sigma=0.0032282548490911722
 avg_lik_loss=2.7123516559600827 avg_pri_loss=14.52400255203247 avg_pred_loss=0.6983350783586503


epoch: 2 train_loss=16.52465124130249 val_outcome_error=2.2269823459045086 val_reconstruction_error=1.4482758620689655
  avg_predicted_sigma=0.0023915565107017756
 avg_lik_loss=2.7917568564414985 avg_pri_loss=14.410702419281005 avg_pred_loss=0.9480383574962615


epoch: 3 train_loss=16.452639007568358 val_outcome_error=0.4389734300637199 val_reconstruction_error=1.3448275862068966
  avg_predicted_sigma=0.0025186894927173853
 avg_lik_loss=2.815321946144104 avg_pri_loss=14.31354360580444 avg_pred_loss=0.9371268868446349


...

epoch: 45 train_loss=15.642750072479247 val_outcome_error=0.6824656529334658 val_reconstruction_error=1.4827586206896552
  avg_predicted_sigma=0.0015829639742150903
 avg_lik_loss=2.379224157333374 avg_pri_loss=13.978937816619872 avg_pred_loss=0.5329810038208962


epoch: 46 train_loss=16.785695171356203 val_outcome_error=18.610887403853276 val_reconstruction_error=1.3103448275862069
  avg_predicted_sigma=0.003350081155076623
 avg_lik_loss=2.7069848299026487 avg_pri_loss=13.914460849761964 avg_pred_loss=5.175672754645348


epoch: 47 train_loss=19.407474327087403 val_outcome_error=0.44719812409966225 val_reconstruction_error=1.4827586206896552
  avg_predicted_sigma=0.0010541637893766165
 avg_lik_loss=4.038321757316589 avg_pri_loss=15.231555557250974 avg_pred_loss=5.7256697654724125


epoch: 48 train_loss=20.735252380371094 val_outcome_error=0.5430267402734252 val_reconstruction_error=4.448275862068965
  avg_predicted_sigma=0.0005790798459202051
 avg_lik_loss=3.8125493764877314 avg_pri_loss=17.679071617126464 avg_pred_loss=1.654706341028214


epoch: 49 train_loss=21.133556747436522 val_outcome_error=0.691004972485829 val_reconstruction_error=3.8448275862068964
  avg_predicted_sigma=0.0010573911713436246
 avg_lik_loss=3.442349910736083 avg_pri_loss=18.579251861572267 avg_pred_loss=1.1077841103076933


In [46]:
model.save('./output/1016-2nd100.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.05,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-2nd100.bin


epoch: 0 train_loss=37.54202499389649 val_outcome_error=32.411007722998875 val_reconstruction_error=7.379310344827586
  avg_predicted_sigma=0.0002382241073064506
 avg_lik_loss=3.9434581756591798 avg_pri_loss=19.25744705200195 avg_pred_loss=11.473155999183655 avg_inv_loss=265.8957717895508


epoch: 1 train_loss=48.871361923217776 val_outcome_error=3.8808335789660813 val_reconstruction_error=12.827586206896552
  avg_predicted_sigma=0.00020587081962730736
 avg_lik_loss=6.442465782165527 avg_pri_loss=27.337408065795902 avg_pred_loss=14.871947574615481 avg_inv_loss=278.4098175048828


epoch: 2 train_loss=53.032846450805664 val_outcome_error=10.563098393733613 val_reconstruction_error=8.068965517241379
  avg_predicted_sigma=0.00022855587303638458
 avg_lik_loss=5.1057000160217285 avg_pri_loss=34.4837745666504 avg_pred_loss=7.355615520477294 avg_inv_loss=280.1660751342773


epoch: 3 train_loss=51.63157539367676 val_outcome_error=2.462198435352008 val_reconstruction_error=3.310344827586207
  avg_predicted_sigma=0.0011839463841170073
 avg_lik_loss=4.399593305587769 avg_pri_loss=36.99230422973633 avg_pred_loss=2.2901225864887236 avg_inv_loss=237.37731323242187


...

epoch: 45 train_loss=33.63150253295899 val_outcome_error=6.241829370752732 val_reconstruction_error=2.6379310344827585
  avg_predicted_sigma=1.00013585324632e-06
 avg_lik_loss=3.224510240554809 avg_pri_loss=20.79139461517334 avg_pred_loss=4.29484429359436 avg_inv_loss=199.8092102050781


epoch: 46 train_loss=33.49901733398437 val_outcome_error=4.2548821270574955 val_reconstruction_error=0.5344827586206896
  avg_predicted_sigma=0.00022997787164058536
 avg_lik_loss=3.207910704612732 avg_pri_loss=20.64258518218994 avg_pred_loss=3.7620539665222164 avg_inv_loss=202.35147705078123


epoch: 47 train_loss=34.2864730834961 val_outcome_error=2.635989280383455 val_reconstruction_error=1.0172413793103448
  avg_predicted_sigma=5.863582555321045e-05
 avg_lik_loss=3.138458919525146 avg_pri_loss=20.63226909637451 avg_pred_loss=2.535417211055756 avg_inv_loss=224.33404235839842


epoch: 48 train_loss=33.70394039154053 val_outcome_error=11.642734054861439 val_reconstruction_error=12.39655172413793
  avg_predicted_sigma=7.332754466915503e-05
 avg_lik_loss=2.7829260587692257 avg_pri_loss=20.44888401031494 avg_pred_loss=3.372390270233154 avg_inv_loss=219.70369415283199


epoch: 49 train_loss=38.3412389755249 val_outcome_error=3.3663238459241267 val_reconstruction_error=3.4310344827586206
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=4.350658583641052 avg_pri_loss=21.526456260681154 avg_pred_loss=12.882030630111695 avg_inv_loss=225.61331634521486


In [47]:
# model.save('./output/1016-3rd005.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.05,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=35.41259994506836 val_outcome_error=3.2736474110197467 val_reconstruction_error=2.4310344827586206
  avg_predicted_sigma=2.780944669211749e-06
 avg_lik_loss=3.242411470413208 avg_pri_loss=22.185314178466793 avg_pred_loss=3.657486498355866 avg_inv_loss=211.05793609619144


epoch: 1 train_loss=37.40574493408203 val_outcome_error=2.5824720113829107 val_reconstruction_error=1.1206896551724137
  avg_predicted_sigma=1.0186948884438607e-06
 avg_lik_loss=3.0918260097503656 avg_pri_loss=22.23122844696045 avg_pred_loss=3.86399347782135 avg_inv_loss=252.11532897949215


epoch: 2 train_loss=35.69974555969239 val_outcome_error=3.7113995672926277 val_reconstruction_error=3.1379310344827585
  avg_predicted_sigma=4.1470346332062036e-05
 avg_lik_loss=3.0405786037445064 avg_pri_loss=21.91698341369629 avg_pred_loss=4.054702681303024 avg_inv_loss=224.20621795654296


epoch: 3 train_loss=35.151196289062504 val_outcome_error=2.3265594773930203 val_reconstruction_error=2.310344827586207
  avg_predicted_sigma=0.00015204651572275907
 avg_lik_loss=2.9693291425704955 avg_pri_loss=21.584529304504393 avg_pred_loss=3.389223527908325 avg_inv_loss=223.4651382446289


epoch: 4 train_loss=33.911288261413574 val_outcome_error=5.590287509011896 val_reconstruction_error=0.9827586206896551
  avg_predicted_sigma=0.000310366740450263
 avg_lik_loss=2.875214886665344 avg_pri_loss=21.084564208984375 avg_pred_loss=4.046979618072511 avg_inv_loss=207.42464447021484


...

epoch: 45 train_loss=30.26681613922119 val_outcome_error=1.6939095743391177 val_reconstruction_error=0.8448275862068966
  avg_predicted_sigma=0.00047857657773420215
 avg_lik_loss=2.655878412723541 avg_pri_loss=17.397884559631343 avg_pred_loss=6.659829449653625 avg_inv_loss=198.70009765625002


epoch: 46 train_loss=30.269083213806148 val_outcome_error=3.561137378454911 val_reconstruction_error=1.8275862068965518
  avg_predicted_sigma=0.0002344052481930703
 avg_lik_loss=2.78002347946167 avg_pri_loss=17.45382080078125 avg_pred_loss=2.539222538471222 avg_inv_loss=211.17237701416013


epoch: 47 train_loss=30.240366172790527 val_outcome_error=11.059030055020475 val_reconstruction_error=1.4655172413793103
  avg_predicted_sigma=0.0002494937798473984
 avg_lik_loss=2.598579621315002 avg_pri_loss=17.53742141723633 avg_pred_loss=4.162388515472412 avg_inv_loss=206.21410827636723


epoch: 48 train_loss=29.986014938354494 val_outcome_error=1.6977092141816124 val_reconstruction_error=0.41379310344827586
  avg_predicted_sigma=0.00021632509015034884
 avg_lik_loss=2.603871726989746 avg_pri_loss=17.563771057128903 avg_pred_loss=2.841186681389809 avg_inv_loss=205.6074325561524


epoch: 49 train_loss=29.579945945739745 val_outcome_error=3.5973906172248173 val_reconstruction_error=1.5172413793103448
  avg_predicted_sigma=4.760428055305965e-05
 avg_lik_loss=2.6004143595695495 avg_pri_loss=17.68765468597412 avg_pred_loss=2.1394177675247192 avg_inv_loss=197.8970863342285


In [48]:
model.save('./output/1016-3rd005.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.10,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-3rd005.bin


epoch: 0 train_loss=41.869728088378906 val_outcome_error=7.761270751682474 val_reconstruction_error=11.827586206896552
  avg_predicted_sigma=3.5632999697554624e-06
 avg_lik_loss=3.058523416519165 avg_pri_loss=17.980169105529786 avg_pred_loss=4.980336785316468 avg_inv_loss=209.2521255493164


epoch: 1 train_loss=44.182183074951176 val_outcome_error=7.392015164204365 val_reconstruction_error=5.0344827586206895
  avg_predicted_sigma=1.000537281470315e-06
 avg_lik_loss=3.879617857933045 avg_pri_loss=20.351381874084467 avg_pred_loss=6.487640857696533 avg_inv_loss=199.1510986328125


epoch: 2 train_loss=48.527642822265626 val_outcome_error=7.1982125817434115 val_reconstruction_error=2.3620689655172415
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.6536529541015623 avg_pri_loss=22.590437126159667 avg_pred_loss=5.061499094963074 avg_inv_loss=226.22392578125002


epoch: 3 train_loss=48.31153526306152 val_outcome_error=3.1886186829822702 val_reconstruction_error=3.9310344827586206
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.6330681800842286 avg_pri_loss=24.457709693908694 avg_pred_loss=3.932622694969177 avg_inv_loss=208.69021453857422


...

epoch: 45 train_loss=48.46989784240723 val_outcome_error=4.272334476349657 val_reconstruction_error=3.7758620689655173
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.634249901771546 avg_pri_loss=25.746514701843264 avg_pred_loss=3.7095131874084473 avg_inv_loss=198.44802551269532


epoch: 46 train_loss=48.85578575134277 val_outcome_error=6.408561934469223 val_reconstruction_error=2.189655172413793
  avg_predicted_sigma=1.0986634606524603e-06
 avg_lik_loss=3.0551563739776615 avg_pri_loss=25.57117652893066 avg_pred_loss=3.935825538635254 avg_inv_loss=209.03880157470704


epoch: 47 train_loss=47.10050582885743 val_outcome_error=2.714355561332155 val_reconstruction_error=1.103448275862069
  avg_predicted_sigma=1.8825650158760254e-06
 avg_lik_loss=3.136118483543396 avg_pri_loss=25.160096359252933 avg_pred_loss=4.73620480298996 avg_inv_loss=193.0829246520996


epoch: 48 train_loss=48.13586349487304 val_outcome_error=4.317130529196567 val_reconstruction_error=2.086206896551724
  avg_predicted_sigma=1.0002000863096328e-06
 avg_lik_loss=3.1327114105224614 avg_pri_loss=25.025212669372564 avg_pred_loss=2.7624495029449463 avg_inv_loss=208.54595794677735


epoch: 49 train_loss=45.44277305603026 val_outcome_error=3.0995798061308246 val_reconstruction_error=3.8620689655172415
  avg_predicted_sigma=2.853883643183508e-06
 avg_lik_loss=3.0565540075302122 avg_pri_loss=24.59779396057129 avg_pred_loss=3.544101619720459 avg_inv_loss=184.89584197998047


In [49]:
# model.save('./output/1016-3rd010.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.10,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=44.79040260314942 val_outcome_error=3.515863280268368 val_reconstruction_error=3.4827586206896552
  avg_predicted_sigma=7.561239908682182e-05
 avg_lik_loss=2.857331943511962 avg_pri_loss=24.08754920959473 avg_pred_loss=3.480081295967102 avg_inv_loss=185.23518905639648


epoch: 1 train_loss=44.933296203613274 val_outcome_error=4.837917509545536 val_reconstruction_error=2.1551724137931036
  avg_predicted_sigma=0.00019049055117648095
 avg_lik_loss=3.193693208694458 avg_pri_loss=23.66446399688721 avg_pred_loss=3.9306565284729005 avg_inv_loss=186.6215026855469


epoch: 2 train_loss=49.7298770904541 val_outcome_error=5.394169735764256 val_reconstruction_error=3.103448275862069
  avg_predicted_sigma=0.00020774974836967885
 avg_lik_loss=2.8566344022750854 avg_pri_loss=23.276808738708496 avg_pred_loss=4.712823283672332 avg_inv_loss=239.96793518066403


epoch: 3 train_loss=46.220312881469724 val_outcome_error=3.616729268160494 val_reconstruction_error=4.0344827586206895
  avg_predicted_sigma=3.793381620198488e-05
 avg_lik_loss=3.2073486328125003 avg_pri_loss=23.195196342468265 avg_pred_loss=4.310557317733764 avg_inv_loss=203.08942031860352


epoch: 4 train_loss=42.9390281677246 val_outcome_error=2.6502978599789193 val_reconstruction_error=2.1206896551724137
  avg_predicted_sigma=0.00014325091615319252
 avg_lik_loss=3.1965159654617312 avg_pri_loss=23.496409225463864 avg_pred_loss=2.568728899955749 avg_inv_loss=170.86762542724608


...

epoch: 45 train_loss=42.12237625122071 val_outcome_error=4.973431752082432 val_reconstruction_error=4.103448275862069
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.9119817972183224 avg_pri_loss=19.950161933898926 avg_pred_loss=7.225085210800171 avg_inv_loss=190.13899307250975


epoch: 46 train_loss=44.23422088623046 val_outcome_error=7.2738644116456275 val_reconstruction_error=4.4655172413793105
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.781525826454163 avg_pri_loss=21.39939250946045 avg_pred_loss=7.92390947341919 avg_inv_loss=187.88519744873048


epoch: 47 train_loss=47.3433765411377 val_outcome_error=3.8637018259461233 val_reconstruction_error=8.448275862068966
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.3961443424224855 avg_pri_loss=22.783290100097656 avg_pred_loss=3.5957593441009528 avg_inv_loss=217.8142158508301


epoch: 48 train_loss=45.865857696533205 val_outcome_error=3.5001616118641303 val_reconstruction_error=4.724137931034483
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.282088160514831 avg_pri_loss=23.559456634521485 avg_pred_loss=3.1301494359970086 avg_inv_loss=197.6443717956543


epoch: 49 train_loss=47.78568153381348 val_outcome_error=11.684767153549066 val_reconstruction_error=1.3793103448275863
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.904794216156006 avg_pri_loss=23.677317619323734 avg_pred_loss=5.350138783454893 avg_inv_loss=215.03801727294922


In [50]:
model.restore('./output/1016-3rd005.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.075,
        max_epochs=50, invar_batchsize=32
    )

INFO:tensorflow:Restoring parameters from ./output/1016-3rd005.bin


Model restored from: ./output/1016-3rd005.bin


epoch: 0 train_loss=36.460785675048825 val_outcome_error=15.436566738998481 val_reconstruction_error=3.689655172413793
  avg_predicted_sigma=2.093207513098605e-05
 avg_lik_loss=2.751442813873291 avg_pri_loss=17.69974765777588 avg_pred_loss=3.8928939104080196 avg_inv_loss=217.11361541748045


epoch: 1 train_loss=35.49798145294189 val_outcome_error=4.804028497939192 val_reconstruction_error=1.6379310344827587
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.0577295303344725 avg_pri_loss=17.97383270263672 avg_pred_loss=3.0187762856483458 avg_inv_loss=199.16616973876953


epoch: 2 train_loss=38.418480682373044 val_outcome_error=2.271434585482276 val_reconstruction_error=2.7413793103448274
  avg_predicted_sigma=1.0000491101891384e-06
 avg_lik_loss=2.9491615533828734 avg_pri_loss=18.516805458068845 avg_pred_loss=5.747381019592284 avg_inv_loss=225.6072746276855


...

epoch: 45 train_loss=38.35516738891601 val_outcome_error=1.7499883365587514 val_reconstruction_error=2.7586206896551726
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.0114565610885617 avg_pri_loss=22.187192726135248 avg_pred_loss=1.948829263448715 avg_inv_loss=187.2223327636719


epoch: 46 train_loss=40.16341781616212 val_outcome_error=2.511366347127715 val_reconstruction_error=2.0689655172413794
  avg_predicted_sigma=0.00016358584980480373
 avg_lik_loss=3.070428943634033 avg_pri_loss=21.779941368103028 avg_pred_loss=2.5722604990005493 avg_inv_loss=214.14533386230465


epoch: 47 train_loss=39.68603477478027 val_outcome_error=10.433007580522327 val_reconstruction_error=6.4655172413793105
  avg_predicted_sigma=2.036120895354543e-05
 avg_lik_loss=2.8268652915954586 avg_pri_loss=21.38026351928711 avg_pred_loss=4.314922618865967 avg_inv_loss=211.4595932006836


epoch: 48 train_loss=38.50656623840332 val_outcome_error=5.477436372982712 val_reconstruction_error=3.5172413793103448
  avg_predicted_sigma=1.4536949493049178e-05
 avg_lik_loss=3.1194648265838625 avg_pri_loss=21.148984146118167 avg_pred_loss=6.06962263584137 avg_inv_loss=190.4573715209961


epoch: 49 train_loss=37.60606727600098 val_outcome_error=3.132189046414843 val_reconstruction_error=1.4137931034482758
  avg_predicted_sigma=9.894426329992712e-05
 avg_lik_loss=3.0618043184280395 avg_pri_loss=21.21018047332764 avg_pred_loss=2.668047642707825 avg_inv_loss=187.12793273925783


In [51]:
# model.save('./output/1016-3rd010.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.075,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=39.461534118652345 val_outcome_error=3.525412198024492 val_reconstruction_error=3.5689655172413794
  avg_predicted_sigma=0.00012650976714212447
 avg_lik_loss=3.037052798271179 avg_pri_loss=21.21694793701172 avg_pred_loss=6.030011773109436 avg_inv_loss=203.47486724853516


epoch: 1 train_loss=38.90510635375977 val_outcome_error=5.173356267322551 val_reconstruction_error=2.4827586206896552
  avg_predicted_sigma=0.0004157942021265626
 avg_lik_loss=3.1224622011184695 avg_pri_loss=21.31016292572021 avg_pred_loss=4.949986362457276 avg_inv_loss=196.5625640869141


epoch: 2 train_loss=39.34444122314453 val_outcome_error=3.148160489559008 val_reconstruction_error=1.8275862068965518
  avg_predicted_sigma=0.0004329090879764408
 avg_lik_loss=2.808786916732788 avg_pri_loss=21.19134654998779 avg_pred_loss=4.15783658027649 avg_inv_loss=209.9297225952148


epoch: 3 train_loss=37.34829139709473 val_outcome_error=3.5053840755542565 val_reconstruction_error=0.8620689655172413
  avg_predicted_sigma=0.00031373801175504923
 avg_lik_loss=2.8036978483200072 avg_pri_loss=21.02796001434326 avg_pred_loss=3.6328704833984373 avg_inv_loss=186.7944839477539


epoch: 4 train_loss=38.985546875000004 val_outcome_error=1.4297915122218308 val_reconstruction_error=1.0517241379310345
  avg_predicted_sigma=0.00017542109708301723
 avg_lik_loss=2.8079617977142335 avg_pri_loss=20.69522914886474 avg_pred_loss=2.6717984318733214 avg_inv_loss=215.24942321777343


...

epoch: 45 train_loss=33.46207733154296 val_outcome_error=3.315751727042908 val_reconstruction_error=2.0517241379310347
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.709380578994751 avg_pri_loss=17.492979621887205 avg_pred_loss=2.5376403570175174 avg_inv_loss=183.7576904296875


epoch: 46 train_loss=35.78046684265137 val_outcome_error=4.966566146488922 val_reconstruction_error=2.086206896551724
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.0670511960983275 avg_pri_loss=17.64539070129394 avg_pred_loss=6.168923354148864 avg_inv_loss=198.8975280761719


epoch: 47 train_loss=37.612015151977545 val_outcome_error=6.2114507988836465 val_reconstruction_error=7.9655172413793105
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.911561799049377 avg_pri_loss=18.132007789611816 avg_pred_loss=3.7795861482620237 avg_inv_loss=225.2504104614258


epoch: 48 train_loss=36.82536945343018 val_outcome_error=5.921527326545004 val_reconstruction_error=4.517241379310345
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.118631196022034 avg_pri_loss=18.74275245666504 avg_pred_loss=5.8518192768096915 avg_inv_loss=199.08939590454102


epoch: 49 train_loss=37.96077499389648 val_outcome_error=5.60138639614621 val_reconstruction_error=1.896551724137931
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.2179088830947875 avg_pri_loss=19.86349983215332 avg_pred_loss=6.133080291748047 avg_inv_loss=198.05329437255858


In [52]:
# model.save('./output/1016-3rd010.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.10,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=45.36195106506348 val_outcome_error=3.332854357027156 val_reconstruction_error=5.862068965517241
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.378058123588562 avg_pri_loss=20.56712951660156 avg_pred_loss=4.885661244392394 avg_inv_loss=216.74471740722655


epoch: 1 train_loss=43.467385864257814 val_outcome_error=5.892009784640444 val_reconstruction_error=6.068965517241379
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.5662312030792234 avg_pri_loss=21.02440242767334 avg_pred_loss=5.17147889137268 avg_inv_loss=191.11768493652343


epoch: 2 train_loss=42.59282035827637 val_outcome_error=4.863678489945762 val_reconstruction_error=2.1379310344827585
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.218759059906006 avg_pri_loss=21.173045539855956 avg_pred_loss=6.417460298538208 avg_inv_loss=181.86478729248046


epoch: 3 train_loss=39.407714462280275 val_outcome_error=1.644109959472436 val_reconstruction_error=5.896551724137931
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.3325164794921873 avg_pri_loss=20.860984611511228 avg_pred_loss=3.2396098971366882 avg_inv_loss=158.0088706970215


epoch: 4 train_loss=39.17004222869873 val_outcome_error=5.123923828518306 val_reconstruction_error=2.1551724137931036
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.0150541067123413 avg_pri_loss=20.68485412597656 avg_pred_loss=3.966276979446411 avg_inv_loss=158.92384719848633


...

epoch: 45 train_loss=39.31764450073243 val_outcome_error=6.233057229073923 val_reconstruction_error=1.5172413793103448
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.6720797061920165 avg_pri_loss=19.631867218017575 avg_pred_loss=4.546444082260132 avg_inv_loss=172.54578094482426


epoch: 46 train_loss=42.127699279785155 val_outcome_error=6.170220339387952 val_reconstruction_error=2.8448275862068964
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.8480478525161743 avg_pri_loss=19.607686424255373 avg_pred_loss=3.932314193248749 avg_inv_loss=200.3853759765625


epoch: 47 train_loss=38.96913528442382 val_outcome_error=4.701891266696177 val_reconstruction_error=1.0862068965517242
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.6657333135604864 avg_pri_loss=19.412739944458007 avg_pred_loss=1.922645670175552 avg_inv_loss=176.24846649169922


epoch: 48 train_loss=43.97575378417969 val_outcome_error=8.72017368131091 val_reconstruction_error=3.2586206896551726
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.1938268423080443 avg_pri_loss=19.31616840362549 avg_pred_loss=4.346514654159546 avg_inv_loss=217.55390319824218


epoch: 49 train_loss=39.62221908569336 val_outcome_error=2.7108978992680424 val_reconstruction_error=7.448275862068965
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.1053667783737184 avg_pri_loss=19.425983047485353 avg_pred_loss=7.945942497253417 avg_inv_loss=166.893701171875


In [53]:
# model.save('./output/1016-3rd010.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.10,
        max_epochs=50, invar_batchsize=32
    )

epoch: 0 train_loss=43.646586227417 val_outcome_error=3.77138645876331 val_reconstruction_error=2.4655172413793105
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.163647270202637 avg_pri_loss=19.670529937744142 avg_pred_loss=5.70933313369751 avg_inv_loss=208.56168746948245


epoch: 1 train_loss=41.13015670776367 val_outcome_error=5.427813541506838 val_reconstruction_error=1.6551724137931034
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.303737330436707 avg_pri_loss=20.08079242706299 avg_pred_loss=6.339718866348266 avg_inv_loss=176.95677185058597


epoch: 2 train_loss=42.32919311523437 val_outcome_error=7.1936178560552095 val_reconstruction_error=1.7241379310344827
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.3252777099609374 avg_pri_loss=20.662776184082027 avg_pred_loss=6.880685186386109 avg_inv_loss=182.17332916259764


epoch: 3 train_loss=42.715005493164064 val_outcome_error=3.0792436585077083 val_reconstruction_error=4.793103448275862
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.1975120306015015 avg_pri_loss=21.04202308654785 avg_pred_loss=4.419962382316588 avg_inv_loss=188.37453384399416


epoch: 4 train_loss=45.38133659362792 val_outcome_error=1.591921124243789 val_reconstruction_error=1.7586206896551724
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.091106390953064 avg_pri_loss=21.437345695495605 avg_pred_loss=8.957456159591676 avg_inv_loss=203.5671920776367


...

epoch: 45 train_loss=39.853379821777345 val_outcome_error=3.2929225058421876 val_reconstruction_error=7.224137931034483
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.1468135356903075 avg_pri_loss=19.974654388427734 avg_pred_loss=6.574275636672974 avg_inv_loss=166.23700866699218


epoch: 46 train_loss=40.801198387145995 val_outcome_error=3.390504201793723 val_reconstruction_error=1.6379310344827587
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.070499134063721 avg_pri_loss=19.988332176208495 avg_pred_loss=3.4022775173187254 avg_inv_loss=182.41024322509762


epoch: 47 train_loss=41.98346366882324 val_outcome_error=6.316394454012595 val_reconstruction_error=8.53448275862069
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.1025810480117793 avg_pri_loss=20.134817886352543 avg_pred_loss=4.80448887348175 avg_inv_loss=189.8399353027344


epoch: 48 train_loss=41.41225204467773 val_outcome_error=3.5445519292451557 val_reconstruction_error=0.5172413793103449
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.080041623115539 avg_pri_loss=20.85140056610107 avg_pred_loss=3.816461873054504 avg_inv_loss=179.43446655273436


epoch: 49 train_loss=41.428635406494145 val_outcome_error=1.7698491229965783 val_reconstruction_error=1.6206896551724137
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.046013331413269 avg_pri_loss=21.433375358581543 avg_pred_loss=4.119611525535583 avg_inv_loss=173.80982971191403


In [54]:
model.save('./output/1016-3rd010.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.15,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-3rd010.bin


epoch: 0 train_loss=48.79420127868652 val_outcome_error=3.687820814090723 val_reconstruction_error=3.4827586206896552
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.051480507850647 avg_pri_loss=21.591624832153325 avg_pred_loss=3.7565136671066286 avg_inv_loss=164.40562667846686


epoch: 1 train_loss=54.42550582885742 val_outcome_error=6.152029278056392 val_reconstruction_error=10.344827586206897
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.4729380607604976 avg_pri_loss=21.65147113800049 avg_pred_loss=8.946249866485596 avg_inv_loss=192.2458831787109


epoch: 2 train_loss=53.273189544677734 val_outcome_error=18.069792713378103 val_reconstruction_error=4.586206896551724
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.595804715156555 avg_pri_loss=22.354087829589847 avg_pred_loss=5.043170356750489 avg_inv_loss=184.33967132568358


epoch: 3 train_loss=56.44697723388673 val_outcome_error=14.537718719449037 val_reconstruction_error=2.8620689655172415
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.5552907705307004 avg_pri_loss=23.534667778015137 avg_pred_loss=8.04056966304779 avg_inv_loss=194.43500823974608


...

epoch: 45 train_loss=51.5349365234375 val_outcome_error=2.012885238719888 val_reconstruction_error=8.017241379310345
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.3423187732696533 avg_pri_loss=20.887226867675786 avg_pred_loss=4.828852689266205 avg_inv_loss=183.9216018676758


epoch: 46 train_loss=50.82921905517578 val_outcome_error=1.3512653239548311 val_reconstruction_error=1.1206896551724137
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.2386423349380493 avg_pri_loss=21.764398002624507 avg_pred_loss=1.9918543815612797 avg_inv_loss=177.9552108764649


epoch: 47 train_loss=50.7243263244629 val_outcome_error=1.348422384759806 val_reconstruction_error=2.706896551724138
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.907623910903931 avg_pri_loss=22.472465324401856 avg_pred_loss=1.316154909133911 avg_inv_loss=175.7342315673828


epoch: 48 train_loss=51.54911308288575 val_outcome_error=3.00482000033609 val_reconstruction_error=1.7758620689655173
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.8445564746856693 avg_pri_loss=22.51075496673584 avg_pred_loss=2.2307554960250853 avg_inv_loss=180.21716384887696


epoch: 49 train_loss=50.59141845703125 val_outcome_error=2.8454651002956206 val_reconstruction_error=4.5344827586206895
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.8327692508697515 avg_pri_loss=22.196403312683106 avg_pred_loss=2.4264265298843393 avg_inv_loss=175.6472244262695


In [55]:
model.save('./output/1016-3rd015.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.20,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-3rd015.bin


epoch: 0 train_loss=61.519417190551756 val_outcome_error=1.1457188913290204 val_reconstruction_error=2.706896551724138
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.0934295892715453 avg_pri_loss=22.179208183288576 avg_pred_loss=3.1098624318838115 avg_inv_loss=184.5617950439453


epoch: 1 train_loss=59.12804450988769 val_outcome_error=2.7608383216750694 val_reconstruction_error=2.8793103448275863
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.224165844917297 avg_pri_loss=22.47212867736817 avg_pred_loss=2.0874397754669194 avg_inv_loss=171.57566757202147


epoch: 2 train_loss=61.199628829956055 val_outcome_error=0.7315981687434501 val_reconstruction_error=1.3275862068965518
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.1324530839920044 avg_pri_loss=23.11471519470215 avg_pred_loss=2.321583199501038 avg_inv_loss=179.09179916381834


epoch: 3 train_loss=62.197625732421876 val_outcome_error=1.8182184163967037 val_reconstruction_error=1.2241379310344827
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.8192786693572995 avg_pri_loss=23.45119686126709 avg_pred_loss=1.9798300623893739 avg_inv_loss=184.29968185424804


...

epoch: 45 train_loss=57.76066207885743 val_outcome_error=3.849321211689243 val_reconstruction_error=3.2758620689655173
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.7899988174438475 avg_pri_loss=21.859815788269046 avg_pred_loss=1.8799917340278625 avg_inv_loss=169.90900878906248


epoch: 46 train_loss=55.6655387878418 val_outcome_error=3.8957542557519704 val_reconstruction_error=4.155172413793103
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.711998617649079 avg_pri_loss=21.730571174621584 avg_pred_loss=3.097380471229553 avg_inv_loss=159.24723205566406


epoch: 47 train_loss=55.57531967163086 val_outcome_error=2.9296428975671316 val_reconstruction_error=1.3103448275862069
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.80150499343872 avg_pri_loss=21.71242694854736 avg_pred_loss=2.91891987323761 avg_inv_loss=158.6287628173828


epoch: 48 train_loss=57.22169189453125 val_outcome_error=1.2943807857480458 val_reconstruction_error=3.2413793103448274
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.630789041519165 avg_pri_loss=21.78734951019287 avg_pred_loss=2.2127333343029028 avg_inv_loss=167.99467315673826


epoch: 49 train_loss=60.548250579833976 val_outcome_error=5.8110782340026 val_reconstruction_error=2.1724137931034484
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.6494103789329526 avg_pri_loss=22.00017127990722 avg_pred_loss=2.0168514132499693 avg_inv_loss=183.71645660400392


In [56]:
model.save('./output/1016-3rd020.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.25,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-3rd020.bin


epoch: 0 train_loss=63.39526863098144 val_outcome_error=7.0747562075991235 val_reconstruction_error=1.6896551724137931
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.9392544269561767 avg_pri_loss=21.956103706359862 avg_pred_loss=5.936926770210267 avg_inv_loss=154.4118438720703


epoch: 1 train_loss=67.53970756530761 val_outcome_error=4.242209001976139 val_reconstruction_error=0.46551724137931033
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.818200969696045 avg_pri_loss=21.934678077697757 avg_pred_loss=4.661435413360596 avg_inv_loss=172.51216735839844


epoch: 2 train_loss=72.25191764831543 val_outcome_error=2.74868382691773 val_reconstruction_error=2.706896551724138
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.0405683040618894 avg_pri_loss=21.83836174011231 avg_pred_loss=4.360404706001281 avg_inv_loss=191.11358032226562


epoch: 3 train_loss=71.70815048217774 val_outcome_error=12.719368556011514 val_reconstruction_error=1.3620689655172413
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.2657079696655273 avg_pri_loss=21.741297721862793 avg_pred_loss=6.830325508117676 avg_inv_loss=186.55188598632813


...

epoch: 45 train_loss=65.23299560546876 val_outcome_error=1.5892371131986678 val_reconstruction_error=1.6379310344827587
  avg_predicted_sigma=0.00011890904715983197
 avg_lik_loss=2.763018274307251 avg_pri_loss=21.378627586364747 avg_pred_loss=2.7304394662380216 avg_inv_loss=167.09339294433596


epoch: 46 train_loss=56.994161605834954 val_outcome_error=1.4148094388240422 val_reconstruction_error=1.9310344827586208
  avg_predicted_sigma=0.00012570906255859882
 avg_lik_loss=2.6241223335266115 avg_pri_loss=21.148505210876465 avg_pred_loss=1.3195933759212493 avg_inv_loss=136.6255905151367


epoch: 47 train_loss=69.34712409973145 val_outcome_error=4.697915030960842 val_reconstruction_error=1.5344827586206897
  avg_predicted_sigma=5.2998213504906744e-05
 avg_lik_loss=2.740440273284911 avg_pri_loss=20.94228916168213 avg_pred_loss=1.5379413008689882 avg_inv_loss=186.2110900878906


epoch: 48 train_loss=64.72436866760253 val_outcome_error=0.5950554155480695 val_reconstruction_error=0.6379310344827587
  avg_predicted_sigma=0.0002598821301944554
 avg_lik_loss=2.9240808010101316 avg_pri_loss=20.7534912109375 avg_pred_loss=3.1104239583015447 avg_inv_loss=166.53006896972653


epoch: 49 train_loss=63.87460632324218 val_outcome_error=3.4180496588008533 val_reconstruction_error=1.0
  avg_predicted_sigma=0.00016651430632919073
 avg_lik_loss=2.7898041248321532 avg_pri_loss=20.56275997161865 avg_pred_loss=2.5697236955165863 avg_inv_loss=164.7819763183594


In [57]:
model.save('./output/1016-3rd025.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.30,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-3rd025.bin


epoch: 0 train_loss=73.23124542236327 val_outcome_error=1.5466062560635863 val_reconstruction_error=4.672413793103448
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.8654086351394654 avg_pri_loss=20.425905609130858 avg_pred_loss=3.3909498929977415 avg_inv_loss=168.17463531494138


epoch: 1 train_loss=67.10992355346679 val_outcome_error=2.5347090249856423 val_reconstruction_error=1.8620689655172413
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.698227190971375 avg_pri_loss=20.235151290893555 avg_pred_loss=3.463318300247192 avg_inv_loss=148.85729904174806


epoch: 2 train_loss=76.87350311279296 val_outcome_error=1.060307579382286 val_reconstruction_error=1.793103448275862
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.193340849876404 avg_pri_loss=20.244592285156255 avg_pred_loss=3.175706791877747 avg_inv_loss=179.98917236328126


epoch: 3 train_loss=73.14817161560059 val_outcome_error=1.6226979178332241 val_reconstruction_error=1.3448275862068966
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.028553009033203 avg_pri_loss=20.235865592956543 avg_pred_loss=2.9964789152145386 avg_inv_loss=168.23575439453123


...

epoch: 45 train_loss=89.55672073364256 val_outcome_error=1.9258506373024984 val_reconstruction_error=2.3620689655172415
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.8786128759384155 avg_pri_loss=24.63040084838867 avg_pred_loss=3.1403574466705324 avg_inv_loss=209.39746551513673


epoch: 46 train_loss=68.69113883972167 val_outcome_error=3.9181681619694677 val_reconstruction_error=2.5172413793103448
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.863274908065796 avg_pri_loss=24.275608634948732 avg_pred_loss=2.4821467399597177 avg_inv_loss=141.4395378112793


epoch: 47 train_loss=78.28802108764648 val_outcome_error=4.828051989525192 val_reconstruction_error=4.5
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.9872983217239377 avg_pri_loss=24.01237392425537 avg_pred_loss=3.9593917846679694 avg_inv_loss=172.92302856445315


epoch: 48 train_loss=77.48545150756837 val_outcome_error=1.2218885473302858 val_reconstruction_error=1.1206896551724137
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.0331678390502925 avg_pri_loss=23.723383522033693 avg_pred_loss=2.6756804943084718 avg_inv_loss=171.8405807495117


epoch: 49 train_loss=73.54503860473633 val_outcome_error=8.10885071894675 val_reconstruction_error=3.0172413793103448
  avg_predicted_sigma=1.0855344044102821e-05
 avg_lik_loss=3.2845803022384636 avg_pri_loss=23.38578033447266 avg_pred_loss=3.496114301681519 avg_inv_loss=158.45288314819337


In [58]:
model.save('./output/1016-3rd030.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.35,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-3rd030.bin


epoch: 0 train_loss=79.29945526123048 val_outcome_error=1.6944413162151524 val_reconstruction_error=2.2586206896551726
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.3769316196441648 avg_pri_loss=23.17059860229492 avg_pred_loss=3.3295484662055967 avg_inv_loss=152.68287048339843


epoch: 1 train_loss=84.83888359069823 val_outcome_error=10.285383045892047 val_reconstruction_error=2.1551724137931036
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.0865077257156375 avg_pri_loss=23.03400573730469 avg_pred_loss=2.95792486667633 avg_inv_loss=169.87304382324214


epoch: 2 train_loss=79.80716552734374 val_outcome_error=3.2639434871015256 val_reconstruction_error=5.948275862068965
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.1787537813186644 avg_pri_loss=22.91643772125244 avg_pred_loss=5.355351352691651 avg_inv_loss=154.2481704711914


epoch: 3 train_loss=76.77553443908693 val_outcome_error=1.2682240554924857 val_reconstruction_error=4.482758620689655
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.232972860336304 avg_pri_loss=22.969906806945797 avg_pred_loss=6.9188672542572025 avg_inv_loss=144.43499603271485


...

epoch: 45 train_loss=84.46603469848633 val_outcome_error=3.6491318000484174 val_reconstruction_error=4.017241379310345
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.974528408050537 avg_pri_loss=24.450120353698733 avg_pred_loss=2.022897255420685 avg_inv_loss=165.7817169189453


epoch: 46 train_loss=82.63741836547852 val_outcome_error=2.803240754403121 val_reconstruction_error=1.6724137931034482
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.2541227340698238 avg_pri_loss=24.196398353576658 avg_pred_loss=1.8726350903511046 avg_inv_loss=160.56943054199218


epoch: 47 train_loss=79.05340270996095 val_outcome_error=3.5743639623432144 val_reconstruction_error=0.9655172413793104
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.795923089981079 avg_pri_loss=23.851649284362793 avg_pred_loss=2.6885386466979986 avg_inv_loss=152.06052703857424


epoch: 48 train_loss=80.92589683532714 val_outcome_error=3.00112349423063 val_reconstruction_error=0.6724137931034483
  avg_predicted_sigma=4.48492392024491e-05
 avg_lik_loss=2.9036839485168455 avg_pri_loss=23.48774528503418 avg_pred_loss=2.585949957370758 avg_inv_loss=158.1621208190918


epoch: 49 train_loss=79.21062393188477 val_outcome_error=2.319491116403297 val_reconstruction_error=2.586206896551724
  avg_predicted_sigma=0.00014613090024795383
 avg_lik_loss=3.0967745780944824 avg_pri_loss=23.337078666687013 avg_pred_loss=2.822302007675171 avg_inv_loss=153.01632080078124


In [59]:
model.save('./output/1016-3rd035.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.40,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-3rd035.bin


epoch: 0 train_loss=94.10614166259764 val_outcome_error=6.891862101130319 val_reconstruction_error=0.46551724137931033
  avg_predicted_sigma=1.0397723144706106e-06
 avg_lik_loss=2.7795269966125487 avg_pri_loss=23.153407478332518 avg_pred_loss=3.939767515659332 avg_inv_loss=171.78051071166993


epoch: 1 train_loss=97.983780670166 val_outcome_error=2.1924015897687044 val_reconstruction_error=0.896551724137931
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.07731397151947 avg_pri_loss=23.15072250366211 avg_pred_loss=4.885150265693665 avg_inv_loss=180.31922912597656


epoch: 2 train_loss=91.62264251708986 val_outcome_error=4.224776241457392 val_reconstruction_error=3.3448275862068964
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.898454022407532 avg_pri_loss=23.605617141723638 avg_pred_loss=3.293796396255493 avg_inv_loss=164.52587890625


epoch: 3 train_loss=84.95428390502931 val_outcome_error=4.440082114328381 val_reconstruction_error=4.379310344827586
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.382759809494019 avg_pri_loss=24.38130989074707 avg_pred_loss=3.229524141550064 avg_inv_loss=144.89338455200195


...

epoch: 45 train_loss=88.16523590087891 val_outcome_error=2.2388675459068486 val_reconstruction_error=1.0862068965517242
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.1307211875915524 avg_pri_loss=24.458163452148437 avg_pred_loss=6.902089214324952 avg_inv_loss=151.57117080688477


epoch: 46 train_loss=89.36765213012694 val_outcome_error=2.882150586725122 val_reconstruction_error=1.7758620689655173
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.118715906143189 avg_pri_loss=24.725914955139157 avg_pred_loss=2.228678810596466 avg_inv_loss=156.21664962768557


epoch: 47 train_loss=96.92122879028321 val_outcome_error=1.9242958806544923 val_reconstruction_error=0.1724137931034483
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.8017320871353153 avg_pri_loss=24.761030387878417 avg_pred_loss=4.7144411325454705 avg_inv_loss=174.5749542236328


epoch: 48 train_loss=85.06363792419434 val_outcome_error=4.111628316480432 val_reconstruction_error=2.7586206896551726
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.184871602058411 avg_pri_loss=24.93054161071777 avg_pred_loss=6.249611926078797 avg_inv_loss=142.88036651611327


epoch: 49 train_loss=89.88032913208006 val_outcome_error=6.618408159832212 val_reconstruction_error=2.0
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.194438552856445 avg_pri_loss=25.245532417297362 avg_pred_loss=8.309825897216799 avg_inv_loss=153.1607810974121


In [60]:
model.save('./output/1016-3rd035.bin')
train_losses, val_outcome_errors, val_seq2seq_errors = model.train(
        train_seqs=train_data[0], train_outcomes=train_data[1],
        test_seqs=test_data[0], test_outcomes=test_data[1],
        which_model='joint', seq2seq_importanceval=0.95,
        kl_importanceval=1.00, invar_importanceval=0.40,
        max_epochs=50, invar_batchsize=32
    )

Model saved as: ./output/1016-3rd035.bin


epoch: 0 train_loss=102.78519973754884 val_outcome_error=6.565815177592691 val_reconstruction_error=4.0
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.252244877815246 avg_pri_loss=25.479770851135253 avg_pred_loss=7.680957174301148 avg_inv_loss=185.03168869018555


epoch: 1 train_loss=90.61389846801758 val_outcome_error=3.4603415952363026 val_reconstruction_error=3.3448275862068964
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.2192584872245793 avg_pri_loss=25.60650424957275 avg_pred_loss=6.113024330139161 avg_inv_loss=155.13460311889648


epoch: 2 train_loss=85.92613296508789 val_outcome_error=2.1461630805771716 val_reconstruction_error=3.0
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.6902529001235957 avg_pri_loss=26.018651962280277 avg_pred_loss=5.830170679092408 avg_inv_loss=141.45371627807617


epoch: 3 train_loss=86.86324691772461 val_outcome_error=10.256573587473378 val_reconstruction_error=2.3275862068965516
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.288159036636352 avg_pri_loss=26.836826896667482 avg_pred_loss=8.916038060188292 avg_inv_loss=141.3247169494629


...

epoch: 45 train_loss=93.814839553833 val_outcome_error=6.266137121857196 val_reconstruction_error=3.413793103448276
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.8057089328765867 avg_pri_loss=21.54319763183594 avg_pred_loss=3.2818564891815183 avg_inv_loss=175.13063354492186


epoch: 46 train_loss=98.25006713867187 val_outcome_error=5.799871112495277 val_reconstruction_error=3.7413793103448274
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.9744077444076535 avg_pri_loss=21.521617317199702 avg_pred_loss=5.251438617706299 avg_inv_loss=184.92237396240236


epoch: 47 train_loss=78.55389099121095 val_outcome_error=2.545792446503963 val_reconstruction_error=1.3448275862068966
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.7471899271011355 avg_pri_loss=21.621563911437985 avg_pred_loss=4.040902757644654 avg_inv_loss=136.5661926269531


epoch: 48 train_loss=85.85071792602538 val_outcome_error=1.5701935693881106 val_reconstruction_error=4.0344827586206895
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=2.9941064834594724 avg_pri_loss=21.947590255737307 avg_pred_loss=2.3398286104202275 avg_inv_loss=154.26534500122068


epoch: 49 train_loss=87.22109375000002 val_outcome_error=8.789557196620834 val_reconstruction_error=2.5344827586206895
  avg_predicted_sigma=1.0000001111620804e-06
 avg_lik_loss=3.1152525782585143 avg_pri_loss=22.122363662719724 avg_pred_loss=5.626721262931825 avg_inv_loss=155.4082374572754


In [62]:
def test_sentence(sentence, model):
    log_alpha = -1000
    init_seq = [get_word_list(sentence)]
    revision_results = \
        model.barrierGradientRevise(init_seq, log_alpha=log_alpha,
                                    outcome_opt_learn_rate=1.0,
                                    max_outcome_opt_iter=1000,
                                    use_adaptive=False)
    x_star, z_star, inferred_improvement, \
    outcome_star, reconstruct_init, z_init, \
    outcome_init, avg_sigma_init, edit_dist = revision_results
    print('{} -> {}'.format(init_seq[0], x_star))


In [63]:
test_sentence('早く寝たい。', model)

iter=0 obj=[[0.]]


iter=100 obj=[[2.071265]]
Elliptical constraint value:[[0.9991731]]
['早く', '寝', 'たい', '。'] -> ['早く', 'お', '風呂', 'に', '入ろう', '。']


C:\Users\elect\PycharmProjects\seq2bseq\model\methods.py:190: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  s1 = [value for value in seq1 if value != '<PAD>']


In [64]:
test_sentence('早く寝たいです。', model)

iter=0 obj=[[0.]]


Elliptical constraint value:[[0.9972693]]
['早く', '寝', 'たい', 'です', '。'] -> ['早く', '寝', 'た', 'ほう', 'が', '良い', 'です', 'ね', '。']


C:\Users\elect\PycharmProjects\seq2bseq\model\methods.py:190: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  s1 = [value for value in seq1 if value != '<PAD>']


In [65]:
test_sentence('それは良い。', model)

iter=0 obj=[[0.]]


iter=100 obj=[[0.8798964]]
Elliptical constraint value:[[0.9978124]]
['それ', 'は', '良い', '。'] -> ['それ', 'は', '良い', 'ね', '。']


C:\Users\elect\PycharmProjects\seq2bseq\model\methods.py:190: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  s1 = [value for value in seq1 if value != '<PAD>']


In [66]:
test_sentence('応援する。', model)

iter=0 obj=[[0.]]
Elliptical constraint value:[[0.43784642]]
['応援', 'する', '。'] -> ['応援', 'し', 'てる', '。']


C:\Users\elect\PycharmProjects\seq2bseq\model\methods.py:190: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  s1 = [value for value in seq1 if value != '<PAD>']


In [67]:
test_sentence('何か不安だなぁ。', model)

iter=0 obj=[[0.]]
Elliptical constraint value:[[0.9968573]]
['何', 'か', '不安', 'だ', 'なぁ', '。'] -> ['何', 'か', '口', 'に', '入れ', 'て', 'は', 'どう', 'しよ', 'う', 'か', '<UNK>']


C:\Users\elect\PycharmProjects\seq2bseq\model\methods.py:190: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  s1 = [value for value in seq1 if value != '<PAD>']


In [70]:
test_sentence('今日は寒かった。', model)

iter=0 obj=[[0.]]


iter=100 obj=[[2.170972]]
iter=200 obj=[[2.1707425]]


iter=300 obj=[[2.17102]]
Elliptical constraint value:[[0.99341893]]
['今日', 'は', '寒かっ', 'た', '。'] -> ['今日', 'は', '寒かっ', 'た', 'です', 'ね', '。']


C:\Users\elect\PycharmProjects\seq2bseq\model\methods.py:190: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  s1 = [value for value in seq1 if value != '<PAD>']
